In [1]:
import os
import pandas as pd
import openai
from openai import OpenAI
import glob 
import json
from datetime import datetime

In [2]:
api_key = "insert_API_Key"

client = OpenAI(api_key=api_key)

In [3]:
data_str = """Line Item	Category	Flavour1	Flavour2	Tag1	Tag2	Tag3	Tag4	Tag5
Heinz By Nature Chicken Dinner 6 Month Plus 120G	Baby Food	Savoury		Health Food Or Drink	Organic Or Additive Free	Microwave Or Oven Ready		
HiPP Organic Vegetables with Rice And Chicken Baby Food Jar 6+ Months 125g	Baby Food	Savoury		Health Food Or Drink	Microwave Or Oven Ready			
SMA Pro 2 Follow-on Milk Powder, 6 mths+ 800g	Baby Food	Neutral		Health Food Or Drink	Microwave Or Oven Ready			
Heinz Sweet Potato & Chicken Jar 120G	Baby Food	Sweet	Savoury	Health Food Or Drink	Microwave Or Oven Ready			
BabyBio Petit Pot Bonne Nuit Carottes Potimarron Riz (Dès 6 Mois) 2x200g	Baby Food	Savoury		Health Food Or Drink	Microwave Or Oven Ready			
Ella's Kitchen Sweet Potato Pumpkin Apple & Blueberries 120G	Baby Food	Sweet	Savoury	On The Go Food Or Drink	Microwave Or Oven Ready			
Heinz By Nature Pasta Bolognese 6 Month Plus 120G	Baby Food	Savoury		Microwave Or Oven Ready	Health Food Or Drink			
Halal Chocolate Baklava 250G	Baked Treats	Sweet		Chocolate	Halal	Middle Eastern Food	Low Nutritional Value	
Fox's Fabulous Salted Caramel Cookies	Baked Treats	Sweet		Low Nutritional Value	On The Go Food Or Drink			
Mammoul with Dates 7 pack	Baked Treats	Sweet		Middle Eastern Food	Low Nutritional Value			
Tesco Chocolate Brownie Bites 20 Pack	Baked Treats	Sweet		On The Go Food Or Drink	Low Nutritional Value	Chocolate		
Tesco Finest Triple Chocolate Cookie 4 Pack	Baked Treats	Sweet		Low Nutritional Value	On The Go Food Or Drink	Premium	Chocolate	
Coop Irresistable Triple Chocolate Cookie 4 Pack	Baked Treats	Sweet		On The Go Food Or Drink	Chocolate	Low Nutritional Value	Premium	
Simply All Butter Shortbread Fingers 250G	Baked Treats	Sweet		On The Go Food Or Drink	Low Nutritional Value	Economy Option		
Mcvitie's Jaffa Cakes 10 Pack	Baked Treats	Sweet		Chocolate	Low Nutritional Value	On The Go Food Or Drink		
Mr Kipling 6 Snowflake Mince Pies	Baked Treats	Sweet		Low Nutritional Value				
Vegan Skinny Double Dough Brownie	Baked Treats	Sweet		Chocolate	Low Calorie Option	Vegan		
Triple Chocolate Cook 200G X 2	Baking Ingredients	Sweet		Chocolate	Low Nutritional Value	Organic Or Additive Free	Cooking Or Baking Ingredient	
Cooks' Ingredients Bicarbonate of Soda	Baking Ingredients	Neutral		Cooking Or Baking Ingredient				
Billington's Demerara Sugar 1kg	Baking Ingredients	Sweet		Cooking Or Baking Ingredient				
Victoria Bakery Plain Flour 1.5Kg	Baking Ingredients	Neutral		Organic Or Additive Free	Premium	Cooking Or Baking Ingredient		
Pride Lemon Juice 250Ml	Baking Ingredients	Sour		Fresh	Cooking Or Baking Ingredient			
Tesco Plain Flour 1.5Kg	Baking Ingredients	Neutral		Cooking Or Baking Ingredient				
Canderel Granular Sweetener 75G	Baking Ingredients	Sweet		Low Calorie Option	On The Go Food Or Drink	Cooking Or Baking Ingredient		
Theos Ready Rolled Filo Pastry 250 G	Baking Ingredients	Neutral		Cooking Or Baking Ingredient				
Dr.Oetker Madagascan Vanilla Extract 35Ml	Baking Ingredients	Sweet		Cooking Or Baking Ingredient				
Tesco Panko Breadcrumbs 150G	Baking Ingredients	Neutral		Cooking Or Baking Ingredient	Japanese Food			
Wholemeal Flour 1kg	Baking Ingredients	Neutral		Health Food Or Drink	Cooking Or Baking Ingredient			
Go Ahead Fruit n Nut Bar Multipack	Bars And Biscuits	Sweet		Health Food Or Drink	Bulk Buy			
Go Ahead Fruit n Nut Bar Lite 200G	Bars And Biscuits	Sweet		Cooking Or Baking Ingredient	Health Food Or Drink	On The Go Food Or Drink	Low Calorie Option	
Morrisons Savers Bourbon Creams Biscuits 296G	Bars And Biscuits	Sweet		Low Nutritional Value	Economy Option			
Eat Natural Fruit & Nut Bar Dark Chocolate with Cranberries and Macadamias 45g	Bars And Biscuits	Sweet		On The Go Food Or Drink	Chocolate	Organic Or Additive Free	Health Food Or Drink	
Snickers Vegan Milk Chocolate Bars 4 X 41.7G	Bars And Biscuits	Sweet		Chocolate	Low Nutritional Value	On The Go Food Or Drink	Vegan	
Fibre One 90 Calorie Salted Caramel Drizzle Squares 5 x 24g (120g)	Bars And Biscuits	Sweet		Low Calorie Option	Low Nutritional Value			
Yorkie Raisin & Biscuit Chocolate Bar 44g	Bars And Biscuits	Sweet		Chocolate	Low Nutritional Value			
Jutrzenka Familijne Chocolate Wafers 180G	Bars And Biscuits	Sweet		Chocolate	Low Nutritional Value	Eastern European Food		
MCVITIES LIGHT RICH TEA BISCUITS 300G	Bars And Biscuits	Sweet		Low Calorie Option	Low Nutritional Value			
Go Ahead Fruit n Nut Bar Lite Multipack	Bars And Biscuits	Sweet		Health Food Or Drink	Bulk Buy	Low Calorie Option		
Coors Light 10X440ml	Beer	Neutral		Low Calorie Option				
Desperados Tequila Flavoured Lager Beer Bottle 650ml	Beer	Savoury	Sweet	On The Go Food Or Drink				
Guinness Draught 4X440ml	Beer	Neutral						
Asahi 660ml	Beer	Neutral		On The Go Food Or Drink				
Guinness 0.0% Draught Stout 4X440ml	Beer	Neutral		Alcohol Free				
Peroni Nastro Azzurro 12X330ml	Beer	Neutral						
Old Speckled Hen Ale Cans 30 X 500Ml	Beer	Neutral		Bulk Buy				
Brewdog Hazy Jane New England Ipa 4X330ml Can	Beer	Neutral						
Carlsberg Alcohol Free 0.0% Lager Beer Bottle 4x330ml	Beer	Neutral		Alcohol Free				
Doom Bar Zero Amber Ale 500Ml	Beer	Neutral		Alcohol Free				
Asda Smart Price Superseeded Loaf with Chilli Flakes	Bread	Spicy	Savoury	Fresh	Economy Option			
Hovis Original 10 Burger Buns 800G	Bread	Neutral		Economy Option	Bulk Buy			
New York Bakery Cinnamon & Raisin Bagels 5Pack	Bread	Sweet		Low Nutritional Value	American Food			
400g Warburtons Danish Lighter Wholemeal Bread	Bread	Neutral		Low Calorie Option	Health Food Or Drink	Fresh		
Old El Paso Regular Super Soft Flour Tortillas 8Pk 326G	Bread	Neutral		Microwave Or Oven Ready	Premium	Mexican Food	Fresh	
Amul Garlic & Herb Paneer Kulcha	Bread	Savoury		Indian Food	Premium	Low Nutritional Value		
Miso Rye Bread 350G	Bread	Savoury		Premium	Low Nutritional Value	Japanese Food		
M&S Mediterranean Vegetable & Pesto Focaccia 370g	Bread	Savoury		Low Nutritional Value	Italian Food	Premium		
Fig, Olive & Cheese Focaccia 350G	Bread	Savoury		Low Nutritional Value	Premium	Animal Produce	Italian Food	
Trio Of Olive Bloomer	Bread	Savoury		Low Nutritional Value	Premium	Fresh	Mediterranean Food	
Haldiram's Soan Papdi Budget Friendly	Cakes	Sweet		Economy Option	Indian Food	Low Nutritional Value		
Matcha Green Tea Cake Roll	Cakes	Sweet		Low Nutritional Value	Japanese Food	Fresh		
Organic Victoria Sponge Mini Muffins	Cakes	Sweet		On The Go Food Or Drink	Organic Or Additive Free			
Soreen Banana Loaf Bar 42G	Cakes	Sweet		On The Go Food Or Drink	Low Nutritional Value			
Sainsbury's Basics Large Chocolate Cake	Cakes	Sweet		Chocolate	Fresh	Economy Option	Low Nutritional Value	
Xbox Celebration Cake	Cakes	Sweet		Low Nutritional Value				
Tesco Finest Sicilian Lemon Curd Muffins 2 Pack	Cakes	Sweet	Sour	Premium	Low Nutritional Value	Fresh		
Essential Iced Fairy Cakes	Cakes	Sweet		Economy Option	Low Nutritional Value			
OGGS Mini Chocolate Cupcakes180g	Cakes	Sweet		Vegan	Low Nutritional Value			
Bosh Luscious Lemon Cake	Cakes	Sour	Sweet	Low Nutritional Value	Vegan	Fresh		
New Covent Garden Soup Co Mushroom Soup 560G	Canned And Preserved Goods	Savoury		Premium	Microwave Or Oven Ready			
Growers Harvest Red Kidney Beans In Water 400G	Canned And Preserved Goods	Savoury		Health Food Or Drink	Cooking Or Baking Ingredient			
Tinned Chilli Con Carne Hot 8 Pack 190G	Canned And Preserved Goods	Savoury	Spicy	Contains Meat	Bulk Buy	Microwave Or Oven Ready	Mexican Food	
Tesco Cream Of Tomato Soup 400G	Canned And Preserved Goods	Savoury		Microwave Or Oven Ready	Economy Option			
Everyday Essentials Mackerel Fillets Sunflower Oil 125G	Canned And Preserved Goods	Savoury		Economy Option	Contains Seafood	Health Food Or Drink		
Blue Dragon Coconut Milk Light 400Ml	Canned And Preserved Goods	Sweet	Savoury	Low Calorie Option	Thai Food	Indian Food	Cooking Or Baking Ingredient	
Green Giant Salt Free Sweetcorn 198G	Canned And Preserved Goods	Sweet		Cooking Or Baking Ingredient	Health Food Or Drink			
Princes Tuna Chunks In Brine 8X145g	Canned And Preserved Goods	Savoury		Bulk Buy	Health Food Or Drink	Contains Seafood		
Spam Chopped Pork And Ham 340G	Canned And Preserved Goods	Savoury		Low Nutritional Value	American Food	Economy Option	Contains Meat	
Tesco Meatballs In Tomato Sauce 395G	Canned And Preserved Goods	Savoury		Microwave Or Oven Ready	Economy Option	Italian Food	Contains Meat	
Tesco Honey Nut Clusters Milk Chocolate Cereal 500G	Cereal	Sweet		Chocolate	Low Nutritional Value			
Tesco Low Fat Special Flakes Cereal 500G	Cereal	Neutral		Health Food Or Drink	Low Calorie Option			
Tesco Special Flakes Red Berry Cereal 375G	Cereal	Sweet		Economy Option	Health Food Or Drink			
Kellogg's Coco Pops 480G	Cereal	Sweet		Chocolate				
Tesco Wheat Biscuits Cereal 48 Pack	Cereal	Neutral		Bulk Buy	Health Food Or Drink			
Tesco Wheat Biscuits Cereal 24 Pack	Cereal	Neutral		Health Food Or Drink				
Kellogg's Special K Original Breakfast Cereal 500g	Cereal	Neutral		Health Food Or Drink	Low Calorie Option			
Quaker Oatso Syrp+1Free Sachet Chocolate Flavour392.	Cereal	Sweet	Chocolate	Microwave Or Oven Ready				
Nestle Curiously Cinnamon 375G	Cereal	Sweet		Low Nutritional Value				
Porridge Oats, SO Organic 750g	Cereal	Neutral		Health Food Or Drink	Organic Or Additive Free	Cooking Or Baking Ingredient		
600g Whole Baking Camenbert with Rosemary	Cheese	Savoury		French Food	Low Nutritional Value	Microwave Or Oven Ready	Animal Produce	
Waitrose: Waitrose Essentials Soy Burrata Balls	Cheese	Savoury		Italian Food	Economy Option	Vegan	Fresh	
Organic Dairylea Dunkers	Cheese	Savoury		Animal Produce	On The Go Food Or Drink	Organic Or Additive Free	Low Nutritional Value	
Vegan Smoked Kashkaval	Cheese	Savoury		Low Nutritional Value	Vegan			
PrÃ©sident French Brie Cheese 200g	Cheese	Savoury		Low Nutritional Value	Fresh	Animal Produce	French Food	
Grated Mozzarella & Cheese Mix 200G	Cheese	Savoury		Low Nutritional Value	Microwave Or Oven Ready	Italian Food	Animal Produce	
Creamfields Soft Animal Free Cheese 200G	Cheese	Neutral		Low Nutritional Value	Fresh	Vegan		
Tesco Finest Parmigiano Reggiano 170 G	Cheese	Savoury		Italian Food	Low Nutritional Value	Premium	Animal Produce	
Seriously Spreadable Original Cheese Spread 125g	Cheese	Savoury		Low Nutritional Value	Animal Produce			
The Laughing Cow Light Cheese Spread Triangles 16Pk 267G	Cheese	Savoury		Animal Produce	On The Go Food Or Drink	Health Food Or Drink	Low Calorie Option	
Rekorderlig Wild Berries Cider x12	Cider	Sweet						
Frosty Jack's Cider 2.5 Litres	Cider	Neutral		Economy Option	Bulk Buy			
K Cider Premium Apple Cider 500ml (24)	Cider	Sweet		Economy Option	Bulk Buy			
All Natural Raspberry Cider 800ml	Cider	Sweet		Organic Or Additive Free	On The Go Food Or Drink			
Strongbow Dark Fruit Cider Can 440ml	Cider	Sweet		On The Go Food Or Drink				
Pulpt Flare Superior South West Cider 500Ml	Cider	Sweet		Premium				
Angioletti Riserva Craft Italian Cider 500ml	Cider	Sweet		Italian Food				
Kopparberg Alcohol Free Strawberry & Lime	Cider	Sweet		Alcohol Free				
Galipette Non-Alcoholic Cidre 330ml	Cider	Sweet		Alcohol Free				
Sainsbury's Vintage Cider, Taste the Difference 500ml	Cider	Sweet		Premium				
Morrisons Savers Cappuccino Coffee Sachets 8x15.5g	Coffee	Neutral		Microwave Or Oven Ready	Economy Option			
Additive Free L'or. Espresso Ristretto Coffee Pods X10 52G	Coffee	Neutral		Organic Or Additive Free	Microwave Or Oven Ready			
Nescafe Gold Cappuccino Coffee Sachets 8x15.5g	Coffee	Neutral		Premium	Microwave Or Oven Ready			
Ueshima House Blend Ground 250g	Coffee	Neutral		Japanese Food	Microwave Or Oven Ready	Premium		
Tin of Grind Coffee	Coffee	Neutral		Microwave Or Oven Ready	Premium			
Beanies Flavour Coffee Creamy Caramel 50g	Coffee	Sweet		Microwave Or Oven Ready				
NiruSpicy Coffee 175g	Coffee	Spicy		Premium	Premium			
ASDA Extra Special Colombian Fairtrade Ground Coffee	Coffee	Neutral		Premium				
Taylors of Harrogate Lazy Sunday Ground Cinnamon andSweetener Coffee	Coffee	Sweet		Low Nutritional Value	Premium			
Illy No Artificials Espresso Beans 500G	Coffee	Neutral		Premium	Premium	Organic Or Additive Free		
Heinz Ketchup Special Extra Large Edition	Condiments And Dressings	Savoury		Bulk Buy				
Tesco Fat Free Vinaigrette Dressing 250Ml	Condiments And Dressings	Savoury		Fresh	Health Food Or Drink	Low Calorie Option		
Winiary Sour Cucumber Relish	Condiments And Dressings	Sour		Eastern European Food				
Low-Calorie Chimichurri Sauce 300G	Condiments And Dressings	Spicy		South American Food	Low Calorie Option			
Hellmann's Light Mayonnaise Squeezy 650Ml	Condiments And Dressings	Savoury		Low Calorie Option	Fresh			
Tesco Red Onion Chutney 295G	Condiments And Dressings	Savoury	Sweet	Cooking Or Baking Ingredient	On The Go Food Or Drink			
Belazu Rose Harissa Paste 130G	Condiments And Dressings	Spicy		Premium	African Food			
Colman's Original English Mustard Jar 100G	Condiments And Dressings	Spicy		British Food				
Colman's Bramley Apple Sauce 155G	Condiments And Dressings	Sweet		British Food				
Deluxe Sweet Chilli & Garlic Sauce 250Ml	Condiments And Dressings	Spicy	Sweet	Chinese Food	Premium			
Sugar Coated Strawberries 0.25KG	Confectionery	Sweet		Fresh				
Freeze at Home Rasberry Tangs (400g)	Confectionery	Sweet		Frozen				
Morrisons The Best All Butter Fudge	Confectionery	Sweet		Low Nutritional Value	Premium			
Haribo Tangfastics 175G	Confectionery	Sweet		On The Go Food Or Drink	Low Nutritional Value	On The Go Food Or Drink		
PICK 'N' MIX     MEDIUM CUP       SALE            	Confectionery	Sweet		Bulk Buy	Low Nutritional Value	On The Go Food Or Drink		
Tesco Strawberry Pencils 75G	Confectionery	Sweet		Low Nutritional Value	On The Go Food Or Drink			
M&M's Crunchy Peanut & Milk Chocolate Bites Pouch Bag 125g	Confectionery	Sweet		On The Go Food Or Drink	Chocolate	Low Nutritional Value		
Parrs Toffee Apples Each	Confectionery	Sweet		Fresh	Low Nutritional Value	On The Go Food Or Drink		
Rowntree's Fruit Pastilles Sweets Sharing Bag 143g	Confectionery	Sweet		Bulk Buy	Low Nutritional Value			
Milky Way Magic Stars Milk Chocolate Bag 100g	Confectionery	Sweet		On The Go Food Or Drink	Chocolate	Low Nutritional Value		
Lidl Simply Chilli Oil (300ml)	Cooking Oils And Vinegars	Spicy		Economy Option	Cooking Or Baking Ingredient			
Cancun Light Avocado Oil with Sea Salt	Cooking Oils And Vinegars	Savoury		Low Calorie Option	Cooking Or Baking Ingredient			Chocolate
Frylight 1 Cal Butter Flavour Cooking Oil Spray	Cooking Oils And Vinegars	Neutral		Low Calorie Option	Cooking Or Baking Ingredient			
Extra Virgin Olive Oil Infused with Truffle 250ml	Cooking Oils And Vinegars	Savoury		Premium	Cooking Or Baking Ingredient			
Napolina Olive Oil 500Ml	Cooking Oils And Vinegars	Neutral		Cooking Or Baking Ingredient	Premium			
Blue Dragon Rice Vinegar 150Ml	Cooking Oils And Vinegars	Sour		Cooking Or Baking Ingredient	Japanese Food	Thai Food	Chinese Food	
Aspall Red Wine Vinegar 350Ml	Cooking Oils And Vinegars	Sour		Cooking Or Baking Ingredient				
Aspall White Wine Vinegar 350Ml	Cooking Oils And Vinegars	Sour		Cooking Or Baking Ingredient				
Organic Balsamic Vinegar Of Modena 250Ml	Cooking Oils And Vinegars	Savoury		Organic Or Additive Free	Cooking Or Baking Ingredient			
Crisp 'N Dry 100% Rapeseed Oil 2 Litre	Cooking Oils And Vinegars	Neutral		Bulk Buy	Cooking Or Baking Ingredient			
Tangy & Salty Plantain Chips 150G	Crisps	Savoury		African Food				
Ziyad Brand Falafel Chips	Crisps	Savoury		Middle Eastern Food				
Mccoy's Cheddar & Onion Grab Bag Crisps 45G	Crisps	Savoury		On The Go Food Or Drink	Low Nutritional Value			
Tyrrells Mixed Root Vegetable Sharing Crisps 125g	Crisps	Savoury		Premium				
Walkers Baked Cheese & Onion Grab Bag Crisps 37.5g	Crisps	Savoury		Low Nutritional Value	On The Go Food Or Drink			
Walkers Sensations Thai Sweet Chilli Sharing Bag Crisps 150g	Crisps	Spicy		Low Nutritional Value	Bulk Buy			
Walkers Max Strong Hot Chicken Wings Crisps150g	Crisps	Spicy	Savoury	On The Go Food Or Drink	Contains Meat	Low Nutritional Value		
Doritos Dippers Cool Original Tortilla Chips 270G	Crisps	Savoury		Mexican Food	Low Nutritional Value			
Kurkure Naughty Tomato Sharing Bag Crisps 100g	Crisps	Savoury		Low Nutritional Value	Indian Food	Bulk Buy		
Popchips Bbq Flavour 5X17g	Crisps	Savoury		Low Calorie Option	Low Nutritional Value			
Grapefruit Flavoured Gummy Vitamin C 200x 20ug	Dietary Supplements	Sour		Health Food Or Drink				
Arla Protein Strawberry Yoghurt With Spoon 200G	Dietary Supplements	Sweet		Health Food Or Drink				
Huel Peanut Flavoured Complete Nutrition Bar 51g	Dietary Supplements	Sweet		On The Go Food Or Drink	Health Food Or Drink			
Moju Fresh Root Ginger Shots 50ml	Dietary Supplements	Spicy		Premium	Health Food Or Drink	On The Go Food Or Drink		
Tesco A-Z Multivitamins & Minerals 90S	Dietary Supplements	Non Food		Health Food Or Drink				
Grenade Cookie Dough Protein Bar 60G	Dietary Supplements	Sweet		On The Go Food Or Drink	Health Food Or Drink			
Gut Gummies	Dietary Supplements	Non Food		Health Food Or Drink				
Cod Liver Oil Softgels	Dietary Supplements	Non Food		Health Food Or Drink				
High Fibre Blend 90 Capsules	Dietary Supplements	Non Food		Health Food Or Drink				
Ufit High Protein Milkshake White Chocolate Flavoured 330Ml	Dietary Supplements	Sweet		On The Go Food Or Drink	Health Food Or Drink			
The Sweet Chilli Houmous 182G	Dips	Sweet	Savoury	Mediterranean Food	On The Go Food Or Drink			
Caramelised Onion Houmous 200G	Dips	Sweet	Savoury	On The Go Food Or Drink	Mediterranean Food			
Taramasalata 200G (C)	Dips	Savoury		On The Go Food Or Drink	Contains Seafood			
Reduced Fat Houmous 200G	Dips	Savoury		Mediterranean Food	Low Calorie Option	On The Go Food Or Drink		
Tzatziki Dip 200G	Dips	Savoury		On The Go Food Or Drink	Mediterranean Food	Animal Produce		
Tesco Mild Salsa Dip 300G	Dips	Savoury		Low Nutritional Value	Mexican Food	On The Go Food Or Drink		
Guacomole 163G	Dips	Savoury		Mexican Food	On The Go Food Or Drink			
Tesco Finest Truffle & Pecorino Mayonnaise Dip 100G	Dips	Savoury		On The Go Food Or Drink	Premium			
Takeaway Garlic & Herb Dip 90g	Dips	Savoury		Low Nutritional Value				
The Foraging Fox Smoked Jalapeno Mayo 240g	Dips	Spicy		Low Nutritional Value				
JUST ESSENTIALS by ASDA 15 Eggs	Eggs	Neutral		Economy Option	Bulk Buy	Cooking Or Baking Ingredient	Animal Produce	
Organic 2 Pack Boiled Eggs ASDA	Eggs	Neutral		Fresh	Organic Or Additive Free	On The Go Food Or Drink	Animal Produce	
Frozen Sour Lime Marinated Eggs 250G	Eggs	Sour		Animal Produce	Frozen			
Clarence Court ostrich egg	Eggs	Neutral		Animal Produce	Cooking Or Baking Ingredient	Premium	Health Food Or Drink	
Tesco Free Range Eggs Mixed Sized 6 Pack	Eggs	Neutral		Fresh	Animal Produce	Health Food Or Drink	Cooking Or Baking Ingredient	
Free & Easy Free From Vegan Egg Replacer 135g	Eggs	Neutral		Cooking Or Baking Ingredient	Vegan	Organic Or Additive Free		
Tesco Finest 6 Free Range Chestnut Maran Eggs	Eggs	Neutral		Animal Produce	Fresh	Premium	Cooking Or Baking Ingredient	
® Aquafaba Vegan Egg Alternative 200ml	Eggs	Neutral		Cooking Or Baking Ingredient	Vegan			
No.1 Free Range Eggs Mixed Sized 6 Pack	Eggs	Neutral	Fresh	Animal Produce	Health Food Or Drink	Premium	Cooking Or Baking Ingredient	
Quails Eggs x12	Eggs	Neutral		Premium	Health Food Or Drink	Cooking Or Baking Ingredient	Animal Produce	
Tesco Finest 2 Thai Prawn Fishcakes 290G	Finfish And Alternatives	Savoury		Premium	Health Food Or Drink	Microwave Or Oven Ready	Contains Seafood	
Counter Tuna Steak	Finfish And Alternatives	Savoury		Fresh	Health Food Or Drink	Premium	Contains Seafood	Cooking Or Baking Ingredient
Ceviche Style Tilapia Fillets - 300G	Finfish And Alternatives	Sour		Cooking Or Baking Ingredient	South American Food	Contains Seafood	Fresh	Health Food Or Drink
Cajun Marinated Seabass Fillets £4/kg	Finfish And Alternatives	Spicy		Premium	Contains Seafood	Cooking Or Baking Ingredient	Health Food Or Drink	Fresh
Tesco 2 Sweet Chilli & Lime Infused Salmon 230G	Finfish And Alternatives	Savoury	Sweet	Fresh	Health Food Or Drink	Cooking Or Baking Ingredient	Microwave Or Oven Ready	Contains Seafood
Vivera Plant Salmon Style Fillets 2X100g	Finfish And Alternatives	Savoury		Fresh	Health Food Or Drink	Cooking Or Baking Ingredient	Vegan	
Plant Chef 8 Fish-Free Goujons 240G	Finfish And Alternatives	Savoury		Microwave Or Oven Ready	Vegan			
The Best 2 Thai Inspired Salmon & Cod F/Cakes 290G	Finfish And Alternatives	Savoury		Thai Food	Premium	Microwave Or Oven Ready	Contains Seafood	
OmniFoods Omni Golden Fillet Plant-Based Beer Battered Fish-Style Fillets	Finfish And Alternatives	Savoury		Vegan	British Food	Microwave Or Oven Ready	Fresh	
Seabasss With Greek Passion Marinade	Finfish And Alternatives	Savoury		Contains Seafood	Mediterranean Food	Fresh	Microwave Or Oven Ready	
Gatorade 12x330ml	Fizzy Pop And Cordial	Sweet		American Food	Bulk Buy	Low Nutritional Value		
Tango Orange Original Cans 6x330ml	Fizzy Pop And Cordial	Sweet		Bulk Buy	Low Nutritional Value			
Mango Frooti 0.5 Litre - Fresh and Juicy - Natural Juices - Refreshing Taste	Fizzy Pop And Cordial	Sweet		Low Nutritional Value	On The Go Food Or Drink	Indian Food		
Inca Kola Lite 330ml	Fizzy Pop And Cordial	Sweet		South American Food	Low Calorie Option	Low Nutritional Value		
Ribena Blackcurrant 850Ml	Fizzy Pop And Cordial	Sweet		Low Nutritional Value				
Pepsi Max No Sugar Cola Bottle 500ml	Fizzy Pop And Cordial	Sweet		On The Go Food Or Drink	Low Calorie Option	Low Nutritional Value		
Diet Coke 2L	Fizzy Pop And Cordial	Sweet		Low Calorie Option	Bulk Buy	Low Nutritional Value		
Ribena Blackcurrant Light Bottle 500 M	Fizzy Pop And Cordial	Sweet		Low Calorie Option	Low Nutritional Value	On The Go Food Or Drink		
Dash Water Sparkling Raspberry 330ml	Fizzy Pop And Cordial	Sweet		Low Calorie Option	Health Food Or Drink	On The Go Food Or Drink		
Whole Earth Organic Sparkling Cola Drink, 24x 330 ml, Flavoured Fizzy Soft Drink Made with Fruit Juice, 100% Natural Ingredients, No Added Refined Sugar, Vegetarian & Vegan, Multi Pack	Fizzy Pop And Cordial	Sweet		Health Food Or Drink	Low Calorie Option	On The Go Food Or Drink	Organic Or Additive Free	
Tesco Pink Lady Apple Min 6 Pack (C)	Fruit	Sweet		Fresh	Health Food Or Drink			
Freezer Ready Banana Smoothie Mix	Fruit	Sweet		Frozen	Health Food Or Drink			
ASDA Frozen for Freshness Summer Fruits	Fruit	Sweet		Health Food Or Drink	Frozen			
Tesco Conference Pear Minimum 4 Pack	Fruit	Sweet		Fresh	Health Food Or Drink			
Tesco Conference Each (C)	Fruit	Sweet		Fresh	Health Food Or Drink			
Suntrail Farms Ripen At Home Peach Minimum 4 Pack	Fruit	Sweet		Health Food Or Drink	Fresh			
Tesco Limes Each	Fruit	Sour		Health Food Or Drink	Cooking Or Baking Ingredient	Fresh		
Perfectly Ripe Gem Avocado Gem Avocado	Fruit	Savoury		Premium	Health Food Or Drink	Fresh		
Tesco Blueberries 250G Plus 100% Ef	Fruit	Sweet		Health Food Or Drink	Fresh			
ASDA Loose Plantain (Typically 350g)	Fruit	Sweet	Savoury	African Food	Fresh	Cooking Or Baking Ingredient	Health Food Or Drink	
SUNNY D SMOOTH ORIGINAL ORANGE CITRUS PUNCH DRINK 26 OZ EACH (1)	Fruit Juices And Smoothies	Sweet		Fresh	Low Nutritional Value			
Innocent Citrus Shield Juice 750Ml	Fruit Juices And Smoothies	Sweet		Fresh	Health Food Or Drink			
Everyday Value 100% Pure Orange Juice Smooth 2 Litre	Fruit Juices And Smoothies	Sweet		Health Food Or Drink	Economy Option	Fresh		
Tropicana Bolivian Pineapple Juice 850Ml	Fruit Juices And Smoothies	Sweet		South American Food	Fresh	Health Food Or Drink		
Tropicana Orange Juice Smooth 300Ml	Fruit Juices And Smoothies	Sweet		On The Go Food Or Drink	Fresh	Health Food Or Drink		
No Added Sugar Cranberry Raspberry Juice Drink 1 Litre	Fruit Juices And Smoothies	Sour		Fresh	Health Food Or Drink	Low Calorie Option		
Innocent Berry Set Go Juice 330Ml	Fruit Juices And Smoothies	Sweet		Fresh	On The Go Food Or Drink			
Tropicana Orange & Mango Juice 1.7 Litres	Fruit Juices And Smoothies	Sweet		Health Food Or Drink	Fresh			
Innocent Bolt From The Blue Juice 750Ml	Fruit Juices And Smoothies	Sweet		Health Food Or Drink	Fresh			
Tropicana Multivitamins Juice 300 Ml	Fruit Juices And Smoothies	Sweet		Health Food Or Drink	On The Go Food Or Drink	Fresh		
Jasmine Rice 1Kg	Grains And Rice	Neutral		Cooking Or Baking Ingredient				
Aldi Red Split Lentils 500G	Grains And Rice	Neutral		Cooking Or Baking Ingredient	Health Food Or Drink			
Pesticide Free Cook Brown Rice 1Kg	Grains And Rice	Neutral		Health Food Or Drink	Organic Or Additive Free	Cooking Or Baking Ingredient		
Bens Original Long Grain Microwave Rice 220g	Grains And Rice	Savoury		Microwave Or Oven Ready				
Worldwide Foods Red & White Quinoa 250g	Grains And Rice	Neutral		Cooking Or Baking Ingredient	Health Food Or Drink			
Tesco Microwave Basmati Rice 250G	Grains And Rice	Savoury		Microwave Or Oven Ready	Indian Food			
Asda Smart Price Cous Cous 500G	Grains And Rice	Neutral		Cooking Or Baking Ingredient	Economy Option			
Tilda Wholegrain Basmati Rice 5kg	Grains And Rice	Neutral		Bulk Buy	Cooking Or Baking Ingredient			
Fullgreen Riced Cauliflower	Grains And Rice	Neutral		Cooking Or Baking Ingredient	Health Food Or Drink			
Five Grains Mix Vegan l GMO Free	Grains And Rice	Neutral		Organic Or Additive Free	Cooking Or Baking Ingredient	Vegan	Health Food Or Drink	
Tagine Spicy Fresh Herb Pack	Herbs, Spices And Flavourings	Spicy		Mediterranean Food	Fresh	Cooking Or Baking Ingredient	Microwave Or Oven Ready	
Cape Herb & Spice Caribbean Jerk Seasoning Rub Tin 100g	Herbs, Spices And Flavourings	Spicy		Jamaican Food	Cooking Or Baking Ingredient	Microwave Or Oven Ready		
Bisto Favourite Gravy Granules 190G	Herbs, Spices And Flavourings	Savoury		British Food	Microwave Or Oven Ready	Cooking Or Baking Ingredient		
Frozen Argentinian Steak Mushroom Flavourings	Herbs, Spices And Flavourings	Savoury		South American Food	Frozen	Cooking Or Baking Ingredient	Microwave Or Oven Ready	
Very Lazy Red Chillies 190G	Herbs, Spices And Flavourings	Spicy		Cooking Or Baking Ingredient	Microwave Or Oven Ready			
Dri-Pak Table Salt 750G	Herbs, Spices And Flavourings	Neutral		Cooking Or Baking Ingredient				
Worcester Sauce 150Ml	Herbs, Spices And Flavourings	Savoury		Cooking Or Baking Ingredient				
Tesco Tahini 300G	Herbs, Spices And Flavourings	Savoury		African Food	Cooking Or Baking Ingredient	Chinese Food	Mediterranean Food	
Thai Dragon Fish Sauce 200Ml	Herbs, Spices And Flavourings	Savoury		Cooking Or Baking Ingredient	Thai Food	Japanese Food	Chinese Food	
Lee Kum Kee Sweet Hoisin Sauce 567G	Herbs, Spices And Flavourings	Sweet	Savoury	Chinese Food	Cooking Or Baking Ingredient			
Ben & Jerry's Cookie Dough S'wich Up Chocolate Ice Cream Tub 465ml	Ice Cream	Sweet		Frozen	Low Nutritional Value	Premium	Animal Produce	
Lemon Sorbet Low-Fat Alternative	Ice Cream	Sour		Low Calorie Option	Animal Produce	Frozen	Low Nutritional Value	
Tesco Finest Raspberry Eton Mess Ice Cream Tub 480ml	Ice Cream	Sweet		Animal Produce	Low Nutritional Value	Premium	Frozen	
Halo Top Animal Free Vanilla Ice Cream Tub 460ml	Ice Cream	Sweet		Vegan	Frozen	Low Calorie Option		
Ben & Jerry's Cookie Dough S'wich Up Vanilla Ice Cream Tub 465ml	Ice Cream	Sweet		Frozen	Chocolate	Low Nutritional Value	Animal Produce	
Haagen-Dazs Fruit Mini Cups Ice Cream 4X95ml	Ice Cream	Sweet		Frozen	Premium	Animal Produce	Low Nutritional Value	Fresh
Vegan Triple Chocolate Ice Cream 900Ml	Ice Cream	Sweet		Frozen	Bulk Buy	Chocolate	Vegan	
Rowntrees Fruit Pastille Lollies 520Ml 100% Extra	Ice Cream	Sweet		Frozen	Low Nutritional Value			Fresh
Kelly's Cornish Clotted Cream Ice Cream 950Ml	Ice Cream	Sweet		Premium	Low Nutritional Value	Bulk Buy		Premium
Ferrero Rocher Classic Ice Cream Sticks 4 X 70Ml	Ice Cream	Sweet		Low Nutritional Value	Chocolate	Premium		
Nurofen 200Mg 16 Tablets	Medicine	Non Food		Untagged				
Tesco Paracetamol Tablets 500Mg 16 Pack	Medicine	Non Food		Untagged				
Sudafed Sinus Max Strength Capsules - 16 Capsules	Medicine	Non Food		Untagged				
Sudafed Sinus Max Strength Capsules 16S	Medicine	Non Food		Untagged				
Nytol Herbal 30S	Medicine	Non Food		Untagged				
Nuromol Dual Action Pain Relief Ibuprofen & Paracetamol 12 Pack	Medicine	Non Food		Untagged				
Dioralyte Blackcurrant 6 Sachets	Medicine	Non Food		Untagged				
Bonjela 15G	Medicine	Non Food		Untagged				
Lemsip Max Cold & Flu Lemon Sachets Paracetamol 10 Pack	Medicine	Non Food		Untagged				
Tesco Allergy & Hayfever Cetirizine 14S	Medicine	Non Food		Untagged				
Everyday Essentials British Semi Oatly 2.272L, 4 Pints	Milk And Milk Drinks	Neutral		Fresh	Economy Option	Vegan	Bulk Buy	
Yeo Valley Organic Skimmed Milk 2 Litre	Milk And Milk Drinks	Neutral		Low Calorie Option	Animal Produce	Fresh	Organic Or Additive Free	
Yeo Valley Organic Whole Milk 2 Litre	Milk And Milk Drinks	Neutral		Animal Produce	Organic Or Additive Free	Fresh		
Sweetened Almond Milk 2 pints	Milk And Milk Drinks	Sweet		Vegan	Fresh			
Chilled Chocolate Flavour Milkshake 400Ml	Milk And Milk Drinks	Sweet		Chocolate	Low Nutritional Value	Animal Produce	Fresh	
Alpro Oat No Sugar Chilled Dairy Free Drink 1L	Milk And Milk Drinks	Neutral		Fresh	Vegan	Health Food Or Drink		
Tesco British Whole Milk 2.272L, 4 Pints	Milk And Milk Drinks	Neutral		Bulk Buy	Fresh	Animal Produce		
Yazoo Strawberry Milkshake 1 Litre Bottle	Milk And Milk Drinks	Sweet		Low Nutritional Value	Animal Produce	Fresh		
St Helen's Farm Semi Skimmed Goats Milk 1L	Milk And Milk Drinks	Neutral		Animal Produce	Premium	Fresh		
Salted Lassi with Bitter Melon	Milk And Milk Drinks	Sour	Savoury	Animal Produce	Indian Food			
Airwaves Menthol & Eucalyptus Sugarfree Chewing Gum 10 Pieces	Mints And Chewing Gum	Neutral						
Wrigley's Extra Strawberry Gum Sugar Free 3 X 12.6G	Mints And Chewing Gum	Sweet						
Fishermans Friend Original 3X25g Pack	Mints And Chewing Gum	Neutral						
Wrigleys Extra Spearmint Chewing Gum 10X12.6G	Mints And Chewing Gum	Neutral		Bulk Buy				
Trebor Extra Strong Peppermint Mint Rolls 4 x 41.3g	Mints And Chewing Gum	Neutral						
Tic Tac Fresh Strong Mint 16.4g	Mints And Chewing Gum	Neutral						
Hubba Bubba Snappy Strawberry Bubblegum Mega Long Tape 56g	Mints And Chewing Gum	Sweet						
100% Natural Mastic Gum from Chios, Greece | Jawline Gum	Mints And Chewing Gum	Neutral		Organic Or Additive Free				
Altoids Curiously Strong Mints	Mints And Chewing Gum	Neutral						
Spearmint Chews (410g)	Mints And Chewing Gum	Neutral		Bulk Buy				
Chocolate Szechuan Peppercorn Martini 250ml	Mixed Drinks And Cocktails	Spicy		Chocolate	On The Go Food Or Drink			
Additive-Free Lychee Martini Cocktail Mix	Mixed Drinks And Cocktails	Sweet		Organic Or Additive Free	Low Nutritional Value			
Chemical Free Low Cal Funkin Cocktails Mixed Pack 8x200ml	Mixed Drinks And Cocktails	Sweet		Low Calorie Option	Organic Or Additive Free	Bulk Buy		
Crabbies Alcoholic Ginger Beer 500Ml x4	Mixed Drinks And Cocktails	Sweet	Spicy					
Bombay Sapphire Gin & Tonic 250ml	Mixed Drinks And Cocktails	Savoury		On The Go Food Or Drink				
Jack Daniels & No Sugar Cola 330Ml	Mixed Drinks And Cocktails	Sweet		On The Go Food Or Drink	Low Calorie Option			
Funkin Nitro Passion Fruit Martini Alcohol Free 4 X 200Ml	Mixed Drinks And Cocktails	Sweet		Alcohol Free				
Sainsbury's Mojito Mocktail Summer Edition, Taste the Difference 750ml	Mixed Drinks And Cocktails	Sweet		Alcohol Free				
Malibu Coconut Rum & Pineapple Pre-Mixed Can	Mixed Drinks And Cocktails	Sweet		On The Go Food Or Drink				
Gordon's Pink 0.0 & Lemonade Can 0% Vol 4x250ml	Mixed Drinks And Cocktails	Sweet		Alcohol Free				
Tiger Tiger Udon Japanese Style Noodles 400G	Noodles	Neutral		Japanese Food	Cooking Or Baking Ingredient			
Nissin Soba Chilli Instant Noodles 92G	Noodles	Spicy		Microwave Or Oven Ready	Low Nutritional Value			
Blue Dragon Whole Wheat Noodles 300G	Noodles	Neutral		Health Food Or Drink	Cooking Or Baking Ingredient			
Waitrose So Healthy Egg Noodles Medium	Noodles	Neutral		Organic Or Additive Free	Cooking Or Baking Ingredient	Health Food Or Drink		
Tiger Tiger Rice Vermicelli 400G	Noodles	Neutral		Chinese Food	Cooking Or Baking Ingredient			
Indonesian Mie Stir Fry Spicy Instant Noodles 80G	Noodles	Spicy		Microwave Or Oven Ready	Indonesian Food			
Sainsbury's Udon Noodles Quick To Cook x2 300g	Noodles	Neutral		Cooking Or Baking Ingredient	Microwave Or Oven Ready			
ThaiTaste Vermicelli Rice Noodles Nests	Noodles	Neutral		Thai Food	Cooking Or Baking Ingredient			
Akagi Joshu Soba Noodles	Noodles	Neutral		Cooking Or Baking Ingredient	Japanese Food			
Vegan Village Vegan Instant Noodles	Noodles	Neutral		Cooking Or Baking Ingredient	Vegan			
Nisa Local Roasted Salted Cashew Nuts 150G	Nuts And Seeds	Savoury		Health Food Or Drink				
Waitrose 1 Cashews 900G	Nuts And Seeds	Savoury		Health Food Or Drink	Premium	Cooking Or Baking Ingredient	Bulk Buy	
Spar Honey Roasted Cashews 150G	Nuts And Seeds	Savoury		Health Food Or Drink				
Grape Tree Chia Seeds	Nuts And Seeds	Neutral		Health Food Or Drink	Cooking Or Baking Ingredient			
Walkers Sensations Thai Sweet Chilli Coated Peanuts Grab Bag 45g	Nuts And Seeds	Savoury	Sweet	On The Go Food Or Drink				
Tesco Coated Peanuts Pigs In Blankets 200G	Nuts And Seeds	Savoury		Low Nutritional Value	On The Go Food Or Drink			
Tesco Flax Seeds 150G	Nuts And Seeds	Neutral		Cooking Or Baking Ingredient	Health Food Or Drink			
ASDA Sunflower Seeds 150g	Nuts And Seeds	Savoury		Cooking Or Baking Ingredient	Health Food Or Drink			
Tesco Walnuts 200G	Nuts And Seeds	Neutral		Health Food Or Drink	Cooking Or Baking Ingredient			
Waitrose Roasted & Salted Mixed Nuts 1kg	Nuts And Seeds	Savoury		Bulk Buy	Health Food Or Drink			
Tesco Egg Custard Tarts 2 Pack	Other Desserts	Sweet		On The Go Food Or Drink	Low Nutritional Value	Animal Produce		
Cartmel Village Shop Sticky Toffee Pudding	Other Desserts	Sweet		Premium	Low Nutritional Value			
Coconut & Rum Mango Sticky Rice	Other Desserts	Sweet		Thai Food				
Gü Free From Spanish Lemon Cheesecakes 2x92g	Other Desserts	Sweet		Animal Produce	Low Nutritional Value	Organic Or Additive Free		
Aero Dark & Milk Mousse 4 X 55G	Other Desserts	Sweet		Low Nutritional Value	Fresh	Chocolate	Animal Produce	
Delicious Desserts Sticky Toffee Eclair 2Pack	Other Desserts	Sweet		On The Go Food Or Drink	Low Nutritional Value			
Abra-Ca-Debora Buttermilk Pancakes 240G	Other Desserts	Sweet		Animal Produce	Microwave Or Oven Ready	Low Nutritional Value		
Tesco Finest 2 Tart Au Citron 150G	Other Desserts	Sour	Sweet	On The Go Food Or Drink	Low Nutritional Value	Premium		
Bonta Divina Tiramisu 500G	Other Desserts	Sweet		Italian Food	Low Nutritional Value			
Tesco Egg Custard Tarts 4 Pack	Other Desserts	Sweet	On The Go Food Or Drink	Low Nutritional Value	On The Go Food Or Drink	Animal Produce	Bulk Buy	
Large Hot Chocolate	Other Hot Beverages	Sweet		Chocolate	Low Nutritional Value			
Horlicks Traditional Malted Milk Drink 500G	Other Hot Beverages	Sweet						
Twinings Spiced Apple Infusion 20 Sachets	Other Hot Beverages	Spicy	Sweet	Health Food Or Drink	Chocolate			
Options Belgian Choc Mint Instant Hot Chocolate 220G	Other Hot Beverages	Sweet		Fresh	Chocolate	Low Nutritional Value		
Ovaltine Original Add Milk 400G	Other Hot Beverages	Sweet		Health Food Or Drink				
Bournvita Malted Drink 500G	Other Hot Beverages	Sweet		Chocolate	Health Food Or Drink			
Swiss Miss Peppermint Hot Cocoa 320G	Other Hot Beverages	Sweet		Fresh	Chocolate			
Boost Chocolate Drink Powder 400G	Other Hot Beverages	Sweet		Low Nutritional Value	Chocolate			
Colman's Hot Mustard Drink 400G	Other Hot Beverages	Spicy						
Ovaltine Light Add Water 300G	Other Hot Beverages	Sweet		Health Food Or Drink	Low Calorie Option			
Horlicks Cardamom Malt Drink 5000G	Other Hot Beverages	Sweet	Spicy	Bulk Buy				
La Famiglia Rana Prosciutto & Mozzarella Tortelloni 250G	Pasta	Savoury		Italian Food	Premium	Animal Produce	Microwave Or Oven Ready	
Whole Wheat Spaghetti	Pasta	Neutral		Italian Food	Cooking Or Baking Ingredient			
Garofalo Fusilli Pasta 500g	Pasta	Neutral		Italian Food	Premium	Cooking Or Baking Ingredient		
ThaiSweet Chili Macaroni Shells 700g	Pasta	Spicy		Thai Food	Cooking Or Baking Ingredient			
Tesco Chicken & Chorizo Fresh Tortelloni 300g	Pasta	Savoury		Contains Meat	Italian Food	Microwave Or Oven Ready	Low Nutritional Value	
Napolina Fusilli Pasta 500G	Pasta	Neutral		Cooking Or Baking Ingredient	Italian Food			
Conchiglie Pasta Shells 1Kg	Pasta	Neutral		Cooking Or Baking Ingredient	Italian Food	Bulk Buy		
Spinach & Ricotta Fresh Tortelloni 300g	Pasta	Savoury		Italian Food	Microwave Or Oven Ready	Health Food Or Drink	Fresh	
Spar Fresh Potato Gnocchi Pasta 400 G	Pasta	Neutral		Fresh	Cooking Or Baking Ingredient	Italian Food		
De Cecco Pasta Conchiglie Rigate 500G	Pasta	Neutral		Cooking Or Baking Ingredient	Italian Food			
All Butter Croissant	Pastries	Sweet		French Food	On The Go Food Or Drink	Low Nutritional Value		
Greek Olive and Feta Puff Bites 200G	Pastries	Savoury		Mediterranean Food	Animal Produce	Low Nutritional Value	On The Go Food Or Drink	
2 Pack Pastel de Nata Kosher Eggs	Pastries	Sweet		On The Go Food Or Drink	Kosher	Mediterranean Food	Animal Produce	
Guava Pastelitos 200G	Pastries	Sweet		South American Food				
Tesco Cinnamon Swirl	Pastries	Sweet		Low Nutritional Value	Fresh	On The Go Food Or Drink		
Tesco Pain Au Chocolat 4 Pack +2 Extra Free	Pastries	Sweet		Low Nutritional Value	Bulk Buy	French Food		
Tesco All Butter Almond Croissant (C)	Pastries	Sweet		French Food	Low Nutritional Value	On The Go Food Or Drink		
Waitrose Essentials Feta And Herb Spinach Pastries 130G	Pastries	Savoury		Mediterranean Food	Low Nutritional Value	Animal Produce	Economy Option	
Specially Selected Maple And Pecan Plait	Pastries	Sweet		On The Go Food Or Drink	Low Nutritional Value	Premium		
Spicy Turkish Meat Pastries	Pastries	Spicy	Savoury	Contains Meat	Turkish Food			
Dreamies Cat Treat Biscuits with Chicken 60g	Pet Food	Savoury		Untagged				
Dreamies Cat Treat Biscuits with Cheese 60g	Pet Food	Savoury		Untagged				
Gourmet Revelations Cat Food Chicken 4X57G	Pet Food	Savoury		Untagged				
Vitakraft Premium Budgie Seed Food 500g	Pet Food	Savoury		Untagged				
Tesco Salmon & Trout Cat Stick Treats 30G	Pet Food	Savoury		Untagged				
Webbox Chicken Lick-E-Lix Cat Treats 5 X15g	Pet Food	Savoury		Untagged				
Tetra Mineral Tropical Fish Flakes 52G	Pet Food	Savoury		Untagged				
Gourmet Mon Petit Fish Selection 6X50g	Pet Food	Savoury		Untagged				
Organic Japanese Pickled Ginger 200G	Pickled Goods	Spicy	Savoury	Organic Or Additive Free	Japanese Food			
Essential Pickled Onions	Pickled Goods	Sour		Economy Option				
Spicy Honey Pickled Jalapeños - Snack Pack	Pickled Goods	Spicy		Mexican Food				
Classic Dill Pickles 500G	Pickled Goods	Sour		American Food				
Tesco Whole Pickled Gherkins 680G	Pickled Goods	Sour		Eastern European Food	Bulk Buy			
Sun Hee Kimchi Spicy Fermented Cabbage 190G	Pickled Goods	Spicy		Korean Food	Health Food Or Drink			
Tesco Pimiento Stuffed Olives In Brine 340G	Pickled Goods	Savoury		Low Nutritional Value	Mediterranean Food			
Tesco Crinkle Cut Gherkins 680G	Pickled Goods	Sour		Bulk Buy	Eastern European Food			
Sarsons Red Cabbage 445g	Pickled Goods	Sour						
Ocean Crown Cooked and Pickled Mussels	Pickled Goods	Savoury	Sour	Contains Seafood				
Goodfellows Honey Roasted Pork with Caramelised Onion	Pizza	Savoury	Sweet	Low Nutritional Value	Italian Food	Contains Meat		
Artisanal Margherita Pizza - Handcrafted with Kosher Ingredients	Pizza	Savoury		Animal Produce	Kosher	Low Nutritional Value	Italian Food	
Tesco Stonebaked Chargrilled Vegetable Pizza Cheese Free 307G	Pizza	Savoury		Vegan	Low Calorie Option	Economy Option	Italian Food	
Tesco Stonebaked Chillies  &Jackfruit  Pepperoni Pizza 277G	Pizza	Spicy	Savoury	Vegan	Italian Food	Low Nutritional Value		
Tesco Finest Ham Mushroom Mascarpone Pizza 450G	Pizza	Savoury		Italian Food	Microwave Or Oven Ready	Premium	Contains Meat	Low Nutritional Value
Chicago Town Large Takeaway Pepperoni Pizza 645G	Pizza	Savoury		Italian Food	Low Nutritional Value	Contains Meat	Microwave Or Oven Ready	
Pizza Express Garlic, Peppers & Soy Mozzarella Pizza 185G	Pizza	Savoury		Italian Food	Low Nutritional Value	Premium	Vegan	
Pizza Express Garlic Dough Balls 95 G	Pizza	Savoury		Microwave Or Oven Ready	Italian Food	Animal Produce	Premium	Low Nutritional Value
Dr. Oetker Mozzarella Ristorante Pizza	Pizza	Savoury		Microwave Or Oven Ready	Economy Option	Low Nutritional Value	Italian Food	Animal Produce
Essential Frozen Pepperoni Pizza	Pizza	Savoury		Economy Option	Microwave Or Oven Ready	Contains Meat	Low Nutritional Value	Italian Food
Rustlers Southern Fried No-Chicken Burger 132G	Poultry And Alternatives	Savoury		Vegan	American Food	Low Nutritional Value	Microwave Or Oven Ready	
Chicken Hearts 500g	Poultry And Alternatives	Savoury		Health Food Or Drink	Cooking Or Baking Ingredient	Fresh	Contains Meat	
Co-op Honest Value Space to Thrive Medium British Whole Kosher Chicken	Poultry And Alternatives	Savoury		Economy Option	Kosher	Fresh	Cooking Or Baking Ingredient	Contains Meat
Frozen Small Turkey Crown 1.5Kg Hormone Free	Poultry And Alternatives	Savoury		Frozen	Cooking Or Baking Ingredient	Contains Meat	Organic Or Additive Free	
Asda Extra Special Chicken Drumsticks Mega Pack 3kg	Poultry And Alternatives	Savoury		Premium	Contains Meat	Bulk Buy	Cooking Or Baking Ingredient	Fresh
Tesco Frozen Small Turkey Crown 1.5Kg	Poultry And Alternatives	Savoury		Frozen	Cooking Or Baking Ingredient	Contains Meat		
Tesco Plant Chef 4 Indian Style No-Chicken Escalopes 340G	Poultry And Alternatives	Savoury		Indian Food	Low Nutritional Value	Vegan	Microwave Or Oven Ready	Cooking Or Baking Ingredient
Halal Breast Bulk	Poultry And Alternatives	Savoury		Halal	Fresh	Cooking Or Baking Ingredient	Bulk Buy	Contains Meat
Irresistible Plant Chef Meat Free Chicken Style 200G	Poultry And Alternatives	Savoury		Vegan	Premium	Low Calorie Option	Cooking Or Baking Ingredient	Fresh
The Vegetarian Butcher What the Cluck Vegan Chicken Chunks 160g	Poultry And Alternatives	Savoury		Cooking Or Baking Ingredient	Vegan	Fresh		
Chopped Grapefruit Slices 200g	Prepared And Dried Fruit	Sour		Health Food Or Drink	Fresh	On The Go Food Or Drink		
Morrisons Chopped Apricots	Prepared And Dried Fruit	Sweet		Health Food Or Drink	On The Go Food Or Drink			
Tesco Mango Chunks 120G	Prepared And Dried Fruit	Sweet		On The Go Food Or Drink	Fresh	Health Food Or Drink		
St Mary's Banana Chips Original 30g	Prepared And Dried Fruit	Sweet		On The Go Food Or Drink				
Dole Peaches In Juice Fruit Pots Multipack 4 x 113g	Prepared And Dried Fruit	Sweet		On The Go Food Or Drink	Health Food Or Drink			
Grower's Harvest Sultanas 500G	Prepared And Dried Fruit	Sweet		Health Food Or Drink	Cooking Or Baking Ingredient			
Tesco Tropical Tray 250G	Prepared And Dried Fruit	Sweet		Fresh	Health Food Or Drink	On The Go Food Or Drink		
Co-op Honest Value Apple Mango, Strawberry & Raspberry 220G	Prepared And Dried Fruit	Sweet		Fresh	On The Go Food Or Drink	Health Food Or Drink	Economy Option	
Sainsbury's Mixed Dried Fruit 1kg	Prepared And Dried Fruit	Sweet		Bulk Buy				
Urban Fruit Pineapple & Coconut 85g	Prepared And Dried Fruit	Sweet		On The Go Food Or Drink	Health Food Or Drink			
Salted & Charred Brussels Sprouts with Lemon - Party Size	Prepared And Dried Vegetables	Savoury	Sour	Bulk Buy	Cooking Or Baking Ingredient	Health Food Or Drink		
Tesco Cooked Beetroot Vacuum Pack 300G (C)	Prepared And Dried Vegetables	Savoury		Fresh	Health Food Or Drink			
Tesco Pepper Stir Fry Mix 320G	Prepared And Dried Vegetables	Savoury		Fresh	Health Food Or Drink	Microwave Or Oven Ready		
Florette Crispy 90G	Prepared And Dried Vegetables	Neutral		Health Food Or Drink	Fresh			
Mushroom Stir Fry Mix 320G	Prepared And Dried Vegetables	Savoury		Cooking Or Baking Ingredient	Health Food Or Drink	Microwave Or Oven Ready	Fresh	
Tesco Carrot Batons 400G	Prepared And Dried Vegetables	Savoury		Fresh	Health Food Or Drink	On The Go Food Or Drink		
Kucharek Dried Vegetables	Prepared And Dried Vegetables	Savoury		Eastern European Food	Cooking Or Baking Ingredient			
Batchelors Bigga Dried Peas	Prepared And Dried Vegetables	Savoury		Health Food Or Drink	Cooking Or Baking Ingredient	Microwave Or Oven Ready		
Sainsbury's Traditionally Sliced Runner Beans 160g	Prepared And Dried Vegetables	Savoury		Microwave Or Oven Ready	Health Food Or Drink	Cooking Or Baking Ingredient		
McCain Ready Baked Jacket Potatoes x4 800g	Prepared And Dried Vegetables	Savoury		Microwave Or Oven Ready				
Yo! Rainbow Salmon Poke 283G	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury		Contains Seafood	Microwave Or Oven Ready	Health Food Or Drink	Japanese Food	
Just Heat Premium Pierogi Platter	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury		Eastern European Food	Premium	Microwave Or Oven Ready		
Strawberry and Light Whipped Cream Sandwich	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Sweet		Japanese Food	On The Go Food Or Drink			
Waitrose & Partners No. 1 Katsu Chicken & Hoisin Duck Sushi 134G	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury		On The Go Food Or Drink	Premium	Contains Meat		
PlantLiving: Falafel & Coriander Houmous Wrap	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury		Vegan	On The Go Food Or Drink			
Tesco Sweet Chilli Chicken Wrap	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury	Sweet	Contains Meat	Fresh	Low Nutritional Value	On The Go Food Or Drink	
Tesco Finest Moroccan Inspired Couscous 230G	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury		African Food	Premium	Health Food Or Drink		
Deluxe Salmon, Prawn & Surimi Rolls Sriracha Mayonnaise 141G	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Spicy	Savoury	Fresh	Health Food Or Drink	On The Go Food Or Drink	Premium	Contains Seafood
Waitose Sausage Bacon & & Egg Triple	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury		On The Go Food Or Drink	Low Nutritional Value	Fresh	Contains Meat	
Wicked Kitchen Sweet Potato Pakora Wrap	Prepared Sandwiches, Sushi, Pasta, Salads And Deli Food	Savoury		Indian Food	On The Go Food Or Drink	Vegan		
Aldi OwnSweet Potato & Chicken Jar 120G	Prepared Sauces And Mixes	Savoury		Economy Option	Microwave Or Oven Ready	Low Nutritional Value		
Free range chocolate cake mix	Prepared Sauces And Mixes	Sweet		Microwave Or Oven Ready	Chocolate	Low Nutritional Value	Organic Or Additive Free	
Plantastic Creamy Pasta SauceSpicy Cajun 350G	Prepared Sauces And Mixes	Spicy		Microwave Or Oven Ready	Italian Food	Low Nutritional Value	Animal Produce	
Fajita Seasoning 30G	Prepared Sauces And Mixes	Spicy		Microwave Or Oven Ready	Mexican Food			
Tesco Tomato Mascarpone Pasta Sauce 500G	Prepared Sauces And Mixes	Savoury	Italian Food	Microwave Or Oven Ready	Animal Produce	Low Nutritional Value		
Homepride Pasta Bake Creamy Tomato & Bacon 485G	Prepared Sauces And Mixes	Savoury		Italian Food	Low Nutritional Value	Microwave Or Oven Ready	Contains Meat	
Tesco Chunky Vegetable Pasta Sauce 500G	Prepared Sauces And Mixes	Savoury		Microwave Or Oven Ready	Low Nutritional Value	Italian Food	Vegan	
Patak's Tikka Masala Curry Sauce 450g	Prepared Sauces And Mixes	Savoury		Microwave Or Oven Ready	Indian Food	Low Nutritional Value		
M&S Pancake Shaker Mix 155g	Prepared Sauces And Mixes	Sweet		Microwave Or Oven Ready	Low Nutritional Value	American Food		
Old El Paso Fajita Dinner Kit 50% Extra Free	Prepared Sauces And Mixes	Spicy		Mexican Food	Microwave Or Oven Ready			
Starbucks Caffe Latte Coffee Drink Venezuela 330Ml	Prepared Tea And Coffee	Sweet		Fresh	South American Food	Animal Produce		
Lipton Ice Tea Peach Bottle 1.25L @so	Prepared Tea And Coffee	Sweet		Fresh				
Jimmy's Low Fat Iced Coffee Mocha 275Ml	Prepared Tea And Coffee	Sweet		Health Food Or Drink	On The Go Food Or Drink	Animal Produce	Low Calorie Option	
Master Kong Green Tea Bottled Drink (500ml x 15 Bottles / 1L x 12 Bottles) 康师傅绿茶	Prepared Tea And Coffee	Neutral		Health Food Or Drink	Japanese Food	Bulk Buy		
Lipton Ice Tea Peach Bottle 500ml	Prepared Tea And Coffee	Sweet		On The Go Food Or Drink				
Starbucks Caramel Macchiato Flavoured Coffee 750Ml	Prepared Tea And Coffee	Sweet		Animal Produce	On The Go Food Or Drink	Fresh		
Regular Flat White + Syrup	Prepared Tea And Coffee	Sweet		On The Go Food Or Drink	Animal Produce	Fresh		
Regular Cappuccino	Prepared Tea And Coffee	Neutral		Fresh	On The Go Food Or Drink	Animal Produce		
Califia Farms Xx Espresso Cold Brew Dairy Free Coffee 750Ml	Prepared Tea And Coffee	Sweet		Fresh	Premium	Vegan	On The Go Food Or Drink	
Regular Cappuccino Oat	Prepared Tea And Coffee	Neutral		Fresh	On The Go Food Or Drink	Vegan		
Hearty Food Co. 4 Breaded Chicken Steaks 505G	Processed Meats And Alternatives	Savoury		Microwave Or Oven Ready	Low Nutritional Value	Contains Meat		
Tesco Finest Chicken Goujons 200G	Processed Meats And Alternatives	Savoury		Premium	Microwave Or Oven Ready	Contains Meat	Low Nutritional Value	
Quorn Vegetarian Chicken Nuggets	Processed Meats And Alternatives	Savoury		Low Nutritional Value	Vegan	Microwave Or Oven Ready		
CORN DOGS Jumbo Bag 9”x 3”.5 100 Piece	Processed Meats And Alternatives	Savoury	Sweet	American Food	Contains Meat	Bulk Buy	Microwave Or Oven Ready	
Tesco 10 Frozen Sweet & Sticky Chicken Kebabs 180G	Processed Meats And Alternatives	Sweet		Contains Meat	Microwave Or Oven Ready	Low Nutritional Value	Frozen	
Tesco Smoked Bacon Lardons 200G	Processed Meats And Alternatives	Savoury		Contains Meat	Low Nutritional Value	Microwave Or Oven Ready		
Richmond 8 Back Bacon Meat Free Rashers 150G	Processed Meats And Alternatives	Savoury		Low Nutritional Value	Microwave Or Oven Ready			
Tesco Smoked Rindless Back Bacon 300G	Processed Meats And Alternatives	Savoury		Microwave Or Oven Ready	Low Nutritional Value	Low Calorie Option	Contains Meat	
Vivera Veggie Greek Kebab 175G	Processed Meats And Alternatives	Savoury		Microwave Or Oven Ready	Vegan	Mediterranean Food		
ASDA Smart Price Tasty 16 Crab Sticks	Processed Meats And Alternatives	Savoury		On The Go Food Or Drink	Contains Seafood			
From Frozen Szechuan Chicken Pasta	Ready Meals	Spicy	Savoury	Contains Meat	Chinese Food	Microwave Or Oven Ready	Frozen	
Tesco Finest Spiced Fruit Lamb Tagine 530g	Ready Meals	Spicy	Savoury	African Food	Contains Meat	Premium	Microwave Or Oven Ready	
Mussel & Prawn Pie Ready Meal 1.5kg	Ready Meals	Savoury		Microwave Or Oven Ready	Contains Seafood	Economy Option	Bulk Buy	
Vegan Quinoa & Vegetable Stir-Fry Bowl - Eco-Friendly Packaging 350G	Ready Meals	Spicy		Vegan	Microwave Or Oven Ready	Health Food Or Drink		
Asda Smart Price Spaghetti Rings In Tomato Sauce 410G	Ready Meals	Savoury		Microwave Or Oven Ready	Economy Option	Italian Food		
Naked Noodle Sweet Chilli Flavour 78G	Ready Meals	Spicy	Savoury	Microwave Or Oven Ready	Low Nutritional Value	Thai Food		
Pot Noodle King Original Curry	Ready Meals	Savoury		Indian Food	Low Nutritional Value			
Tesco Free From Katsu Chicken Curry Rice Ready Meal 400G	Ready Meals	Savoury		Contains Meat	Japanese Food	Microwave Or Oven Ready		
Tesco Chinese Style Chicken Curry With Egg Fried Rice 400G	Ready Meals	Savoury		Chinese Food	Contains Meat	Microwave Or Oven Ready	Low Nutritional Value	
Tesco Thai Green Chicken Curry & Jasmine Rice 400G	Ready Meals	Savoury		Thai Food	Microwave Or Oven Ready	Contains Meat	Low Nutritional Value	
Quick Brai Boerewors Sausages x6	Red Meat And Alternatives	Savoury		African Food	Contains Meat	Low Nutritional Value	Microwave Or Oven Ready	
Basil and Thyme 12 Beef Meatballs 360G	Red Meat And Alternatives	Savoury		Italian Food	Microwave Or Oven Ready	Contains Meat		
Lemongrass & Kaffir Lime Microwaveable Sausages	Red Meat And Alternatives	Savoury		Contains Meat	Microwave Or Oven Ready	Thai Food		
Wicked Kitchen 2 Jalapeno Griller Patties 226G	Red Meat And Alternatives	Spicy		Vegan	Microwave Or Oven Ready	American Food	Low Nutritional Value	
Tesco Beef Lean Steak Mince 5% Fat 500G	Red Meat And Alternatives	Savoury		Fresh	Contains Meat	Low Calorie Option	Cooking Or Baking Ingredient	
Simon Howie Black Pudding 200G	Red Meat And Alternatives	Savoury		Contains Meat	Cooking Or Baking Ingredient	Health Food Or Drink		
Certified Angus Beef Ribeye Steak 800g	Red Meat And Alternatives	Savoury		Fresh	Premium	Contains Meat	Cooking Or Baking Ingredient	
Sainsbury's British Pork Sliced Liver	Red Meat And Alternatives	Savoury		Fresh	Contains Meat	Health Food Or Drink	Cooking Or Baking Ingredient	
Tesco Lean Pork Mince 5% Fat 500G	Red Meat And Alternatives	Savoury		Contains Meat	Cooking Or Baking Ingredient	Low Calorie Option	Health Food Or Drink	
Linda Mccartney 2 Mozzarella Burgers 227G	Red Meat And Alternatives	Savoury		Low Nutritional Value	Vegan	Microwave Or Oven Ready		
ASDA Smart Price Tasty 16 Crab Legs	Shellfish And Alternatives	Savoury		Economy Option	Low Nutritional Value	Low Calorie Option	Contains Seafood	
Tesco Finest Oysters 360G	Shellfish And Alternatives	Savoury		French Food	Premium	Contains Seafood	Cooking Or Baking Ingredient	
Razor Clams 1kg	Shellfish And Alternatives	Savoury		Health Food Or Drink	Fresh	Contains Seafood	Cooking Or Baking Ingredient	
Tesco Microwaveable Organic King Prawns Tandoori Style	Shellfish And Alternatives	Spicy		Indian Food	Contains Seafood	Organic Or Additive Free	Microwave Or Oven Ready	
Tesco Raw Scallops 150G	Shellfish And Alternatives	Savoury		Contains Seafood	Health Food Or Drink	Cooking Or Baking Ingredient	Fresh	
Taste the Difference Mussel 150G	Shellfish And Alternatives	Savoury		Health Food Or Drink	Cooking Or Baking Ingredient	Fresh	Premium	Contains Seafood
Happiee! Plant-Based Shrimpiee 180g	Shellfish And Alternatives	Savoury		Vegan	Health Food Or Drink	On The Go Food Or Drink	Fresh	Cooking Or Baking Ingredient
Tesco Raw Seafood Selection 350G	Shellfish And Alternatives	Savoury		Contains Seafood	Cooking Or Baking Ingredient	Health Food Or Drink	Fresh	
Kingfisher Jumbo Crab 145G	Shellfish And Alternatives	Savoury		Fresh	Contains Seafood	Cooking Or Baking Ingredient	Health Food Or Drink	
Kingfisher Jumbo Not Crab 145G	Shellfish And Alternatives	Savoury		Fresh	Cooking Or Baking Ingredient	Health Food Or Drink	Vegan	
Tesco Value Homestyle Straight Cut Chips 1Kg	Side Dishes	Savoury		Frozen	Microwave Or Oven Ready	Low Nutritional Value	Economy Option	
Mccain Crispy French Fries 1Kg	Side Dishes	Savoury		Low Nutritional Value	Frozen	Microwave Or Oven Ready		
Mccain Crispy French Fries 3Kg	Side Dishes	Savoury		Microwave Or Oven Ready	Low Nutritional Value	Frozen	Bulk Buy	
Multipack Extra Large Vegetable Samosa 98G	Side Dishes	Savoury		Microwave Or Oven Ready	Bulk Buy	Indian Food		
Waitrose Frozen Beer Battered Onion Rings	Side Dishes	Savoury		American Food	Low Nutritional Value	Frozen	Microwave Or Oven Ready	
Tesco Garlic Baguette 205G	Side Dishes	Savoury		Italian Food	Low Nutritional Value	Microwave Or Oven Ready		
Tesco Finest Garlic & Extra Virgin Olive Oil Flatbread 200G	Side Dishes	Savoury		Premium	Italian Food	Low Nutritional Value	Microwave Or Oven Ready	
Paxo Sage & Onion Stuffing Mix 170g	Side Dishes	Savoury		British Food	Low Nutritional Value	Microwave Or Oven Ready		
Patak's Plain Naan Breads 2 per pack	Side Dishes	Savoury		Indian Food	Microwave Or Oven Ready			
itsu vegetable fusion gyoza family pack 1kg	Side Dishes	Savoury		Chinese Food	Microwave Or Oven Ready	Frozen		
Trail Mix Packs 6x150g	Snacks	Sweet		Economy Option	Health Food Or Drink	On The Go Food Or Drink		
The Gym Kitchen Piri Piri Grains 250g	Snacks	Savoury		Health Food Or Drink	On The Go Food Or Drink			
Tao Kae Noi, Crispy Seaweed Snack, 1.12 oz (32 g)	Snacks	Savoury		Japanese Food	On The Go Food Or Drink	Health Food Or Drink		
Weightwatchers lemon & lime chocolate covered raisins	Snacks	Sweet		Low Calorie Option	Chocolate			
Tao Kae Noi Tempura Seaweed Original 40G	Snacks	Savoury		Japanese Food	On The Go Food Or Drink	Health Food Or Drink		
Sainsbury's Selection Charcuterie Platter Slices x37 200g	Snacks	Spicy		Premium	On The Go Food Or Drink	Contains Meat		
Peperami Original Salami 5x20g	Snacks	Savoury		Low Nutritional Value	American Food	On The Go Food Or Drink	Contains Meat	
Billy Brazzier's Smoked Salmon Jerky 85G	Snacks	Savoury		On The Go Food Or Drink	Contains Seafood			
RYVITA Cracked Black Pepper Crispbread 200g	Snacks	Savoury		Health Food Or Drink				
Tesco Small Melton Mowbray Pork Pie 145G	Snacks	Savoury		Contains Meat	On The Go Food Or Drink	Low Nutritional Value		
Spicy Roasted Chickpeas and Quinoa	Snacks	Spicy		Health Food Or Drink	On The Go Food Or Drink	Vegan		
Bombay Mix	Snacks	Savoury		Indian Food	On The Go Food Or Drink			
Quorn Fridge Raiders ©?#	Snacks	Savoury		On The Go Food Or Drink	Low Calorie Option	Vegan		
Dried Mushroom Slices - 65G	Snacks	Savoury		On The Go Food Or Drink	Vegan			
Magicorn EazyPop M/wave Popcorn Sltd Flavour 85g	Snacks	Savoury		Low Nutritional Value	Microwave Or Oven Ready	Low Calorie Option		
Chicken Slice	Snacks	Savoury		On The Go Food Or Drink	Low Nutritional Value	Contains Meat		
Tequila Jelly - Party Pack	Spirits	Spicy		Bulk Buy				
Casamigos Blanco Tequila	Spirits	Neutral		Premium				
Campari Aperitivo Liquer 1L	Spirits	Neutral						
PALLINI Limoncello 50cl	Spirits	Sweet						
Bailey's Original Irish Cream Liqueur Bottle 17% Vol 1L	Spirits	Sweet						
Smirnoff No.21 Red Label Vodka Bottle 0% Vol 1L	Spirits	Neutral		Alcohol Free				
Beefeater Blood Orange Gin 70Cl	Spirits	Sweet						
Tanqueray Flor de Sevilla Alcohol Free 0% Spirit 70cl	Spirits	Neutral		Alcohol Free				
Captain Morgan Original Spiced Gold Rum Based Spirit Bottle 35% 1L	Spirits	Neutral						
Gordon's Alc Free Prm Pink Dist Gin 70cl	Spirits	Sweet		Alcohol Free				
Monster Energy Ultra Choc 9 X 500Ml	Sports And Energy Drinks	Sweet		Chocolate	Low Nutritional Value	Bulk Buy		
Sainsbury's: Sainsbury's Basics Freezer Ready Isotonic Chilli Drinks	Sports And Energy Drinks	Spicy		Frozen	Economy Option	Low Nutritional Value		
Red Bull Energy Drink 6 X 250Ml	Sports And Energy Drinks	Sweet		Low Nutritional Value	On The Go Food Or Drink	Bulk Buy		
Lucozade Energy Orange 900Ml	Sports And Energy Drinks	Sweet		Low Nutritional Value	On The Go Food Or Drink			
Tenzing Natural Energy Drink 250Ml	Sports And Energy Drinks	Sweet		Low Nutritional Value	On The Go Food Or Drink			
Prime Hydration Meta Moon Flavoured 500Ml	Sports And Energy Drinks	Sweet		Premium	On The Go Food Or Drink			
Red Bull The Red Edition Watermelon 355Ml	Sports And Energy Drinks	Sweet		On The Go Food Or Drink	Low Nutritional Value			
Purdey's Rejuvenate Sparkling Grape & Apple with Ginseng 330ml	Sports And Energy Drinks	Sweet		On The Go Food Or Drink	Health Food Or Drink	Low Nutritional Value		
HOLY Energy® STRAWBERRY SHARK	Sports And Energy Drinks	Sweet		Low Nutritional Value				
Grenade ENERGY DRINK BUNDLE (2 FOR £30)	Sports And Energy Drinks	Sweet		Low Nutritional Value	Bulk Buy			
Value PG Tips Gold Tea Bags x70 203g	Tea	Neutral		Economy Option	Health Food Or Drink			
Roibos Teabags 24 100g	Tea	Neutral		Premium	Health Food Or Drink			
Yogi Chocolate Chai Teabags	Tea	Sweet		Chocolate				
Pukka Lemon Ginger Manuka Honey 20Herbal Tea Sachets	Tea	Sour		Premium	Health Food Or Drink			
Tesco 240 Teabags 750G	Tea	Neutral		Bulk Buy	Economy Option			
Tesco Decaffeinated 80 Teabags 250G	Tea	Neutral		Economy Option	Health Food Or Drink			
English Rose Alice in Wonderland Mini Caddy	Tea	Neutral		Premium				
Tesco 240 Teabags 750G	Tea	Neutral		Economy Option	Bulk Buy			
Peppermint 40 Tea Bags 60G	Tea	Neutral		Health Food Or Drink				
Clipper Organic 80 Tea Bags	Tea	Neutral		Health Food Or Drink	Organic Or Additive Free			
Prosecco & Chocolate Hazelnut Spread	Toast Spreads	Sweet		Italian Food	Chocolate	Low Nutritional Value		
Nutella Hazelnut Chocolate Spread 350G	Toast Spreads	Sweet		Low Nutritional Value	Chocolate			
Marmite Yeast Extract Spread 250G	Toast Spreads	Savoury		Organic Or Additive Free	Vegan			
Tesco Crunchy Peanut Butter Low Calorie 900G	Toast Spreads	Savoury		Vegan	Bulk Buy	Low Calorie Option		
Tesco Crunchy Peanut Butter 700G	Toast Spreads	Savoury		Economy Option				
Lurpak Slightly Salted Spreadable Blend of Butter and Rapeseed Oil 250g	Toast Spreads	Savoury		Low Nutritional Value	Cooking Or Baking Ingredient			
Flora Plant Butter Salted 200g	Toast Spreads	Savoury		Cooking Or Baking Ingredient	Vegan	Low Nutritional Value		
Lotus Biscuit Spread Smooth 400G	Toast Spreads	Sweet		Low Nutritional Value				
Bonne Maman Strawberry Conserve370g+20%	Toast Spreads	Sweet		Low Nutritional Value	Premium			
Lyle's Golden Syrup Breakfast 340G	Toast Spreads	Sweet		Low Nutritional Value				
The Tofoo Co. Organic Silken Tofu 250G	Tofu And Other Meat Alternatives	Neutral		Cooking Or Baking Ingredient	Vegan	Health Food Or Drink	Organic Or Additive Free	
The Tofoo Co. Naked Xl Extra Firm Organic Tofu 450G	Tofu And Other Meat Alternatives	Neutral		Cooking Or Baking Ingredient	Organic Or Additive Free	Health Food Or Drink	Vegan	
Satono Yuki Tofu 300G	Tofu And Other Meat Alternatives	Neutral		Vegan	Health Food Or Drink	Japanese Food	Cooking Or Baking Ingredient	
The Tofoo Co Naked Tofu 280G	Tofu And Other Meat Alternatives	Neutral		Vegan	Cooking Or Baking Ingredient	Health Food Or Drink		
Tiba Tempeh Block Organic	Tofu And Other Meat Alternatives	Neutral		Organic Or Additive Free	Health Food Or Drink	Vegan	Cooking Or Baking Ingredient	
Marigold Mo-Du Braised Seitan	Tofu And Other Meat Alternatives	Neutral		Cooking Or Baking Ingredient	Vegan			
Seitan vegan meat substitute wheat protein 150 g INTENSON	Tofu And Other Meat Alternatives	Neutral		Vegan	Cooking Or Baking Ingredient			
Better Nature Mediterranean Tempeh Pieces 180g	Tofu And Other Meat Alternatives	Neutral		Cooking Or Baking Ingredient	Mediterranean Food	Vegan	Health Food Or Drink	
Cauldron Extra Firm Tofu 220g	Tofu And Other Meat Alternatives	Neutral		Cooking Or Baking Ingredient	Vegan			
Quorn Vegetarian Pieces	Tofu And Other Meat Alternatives	Neutral		Vegan	Cooking Or Baking Ingredient			
Asda Smart Price Soda Water 1 Litre	Tonic And Flavoured Water	Neutral		Economy Option				
Fever Tree Rhubarb & Raspberry Tonic Water 16x75cl	Tonic And Flavoured Water	Sweet		Premium	Bulk Buy			
Sainsbury's Basics Mandarin & Lime Tonic Water	Tonic And Flavoured Water	Sour		Economy Option				
Iranian Elderflower Tonic Water 100cl	Tonic And Flavoured Water	Sweet		Middle Eastern Food	Premium			
Tesco Low Calorie American Ginger Ale 1 Litre	Tonic And Flavoured Water	Spicy		Low Calorie Option				
Schweppes Soda Water 1 Litre	Tonic And Flavoured Water	Neutral						
Fever-Tree Refreshingly Light Sweet Rhubarb & Raspberry Tonic Water 500ml	Tonic And Flavoured Water	Sweet		Premium				
Aqua Libra Sparkling Water Infused with Cucumber Mint & Lime	Tonic And Flavoured Water	Sour						
Volvic Touch of Fruit Summer Fruits Still Flavoured Water 2.5L	Tonic And Flavoured Water	Sweet		Bulk Buy				
Eden Bay Tonic Co Premium Mediterranean Tonic Water	Tonic And Flavoured Water	Neutral		Premium	Mediterranean Food			
Tesco Folding Camping Chair Burgundy	Uncategorised	Non Food		Untagged				
Chair	Uncategorised	Non Food		Untagged				
Head & Shoulders 2 In 1 Classic Clean Shampoo & Conditioner 225ml	Uncategorised	Non Food		Untagged				
Royal Flush Playing Cards	Uncategorised	Non Food		Untagged				
Mint Birthday Card Ha Pea Birthday	Uncategorised	Non Food		Untagged				
Andrex Classic Clean Toilet Tissue 9 Mega Rolls	Uncategorised	Non Food		Untagged				
Gillette Fusion 5 Ultra Sensitive Shaving Gel 200Ml	Uncategorised	Non Food		Untagged				
Baylis & Harding Jojoba & Silk Hand Wash 500Ml	Uncategorised	Non Food		Untagged				
Plate Set	Uncategorised	Non Food		Untagged				
Tesco Small Disposable Charcoal Bbq Tray	Uncategorised	Non Food		Untagged				
Tesco Luxury Soft Coconut Oil X4	Uncategorised	Non Food		Untagged				
Paper Salad Everyday Thank You So Much! JA21186	Uncategorised	Non Food		Untagged				
Whole Broccoli 350g	Vegetables	Savoury		Cooking Or Baking Ingredient	Health Food Or Drink	Fresh		
Kambuzi	Vegetables	Spicy		Health Food Or Drink	Fresh	Cooking Or Baking Ingredient		
Tesco Dudhi Each	Vegetables	Neutral		Indian Food	Fresh	Health Food Or Drink		
Mixed Frozen Vegetables - Value Pack	Vegetables	Neutral		Frozen	Economy Option	Health Food Or Drink	Cooking Or Baking Ingredient	
Tesco Loose Red Peppers(C)	Vegetables	Savoury		Fresh	Health Food Or Drink	Cooking Or Baking Ingredient		
Tesco Express Green Beans 220G	Vegetables	Savoury		Health Food Or Drink	Cooking Or Baking Ingredient	Fresh		
Tesco Aubergine Each 250G (C)	Vegetables	Savoury		Health Food Or Drink	Cooking Or Baking Ingredient	Fresh		
Nightingale Farms Cherry Tomatoes 250G	Vegetables	Savoury		Fresh	Health Food Or Drink			
Tesco Whole Cucumber Each	Vegetables	Neutral		Health Food Or Drink	Fresh			
Tesco Portobello Flat Mushrooms 150G	Vegetables	Savoury		Health Food Or Drink	Cooking Or Baking Ingredient	Fresh		
Tesco Ashbeck Still Water 550ml 6 Pack	Water	Neutral		Bulk Buy				
Juicy Water Lemon & Lime Drink 420 Ml	Water	Sour		On The Go Food Or Drink				
Deluxe Taste The Difference Salt Water (250ml)	Water	Savoury		On The Go Food Or Drink	Premium			
San Pellegrino Sparkling Natural Mineral Water 500ml	Water	Neutral		Premium	On The Go Food Or Drink			
Get More Vitamins Recovery Cranberry 500Ml	Water	Sour		Health Food Or Drink	Premium	On The Go Food Or Drink		
Volvic Natural Bottled Mineral Still Water 1L	Water	Neutral		On The Go Food Or Drink				
Actiph Alkaline Ionised Water 600Ml	Water	Neutral		Health Food Or Drink	On The Go Food Or Drink	Premium		
Tesco Sparkling Water Apple & Raspberry 1 Litre	Water	Sweet		On The Go Food Or Drink				
Tesco Elmhurst English Natural Mineral Sparkling Water 2L	Water	Neutral		Bulk Buy				
The Ice Co. Premium Ice Cubes 1Kg	Water	Neutral		Premium				
Fre White Zinfandel Alcohol-Removed Wine	Wine	Sweet		Alcohol Free				
Barefoot Californian Zinfandel 750ml	Wine	Neutral		Economy Option				
Strawberry Linga Wine 75CL	Wine	Sweet						
McGuigan Zero Rosé	Wine	Neutral		Alcohol Free				
Veuve Clicquot Yellow Label Brut Champagne	Wine	Neutral		Premium				
Dark Bordeaux Wine	Wine	Neutral						
Condesa De Flores Cava Brut 75Cl	Wine	Neutral						
Mud House Chile Sauvignon Blanc Rose 750Ml	Wine	Neutral						
Moet & Chandon Brut Imperial Non Vintage Champagne 75Cl	Wine	Neutral		Premium				
Nozeco 75cl	Wine	Neutral		Alcohol Free				
Tesco Soured Cream 300Ml	Yoghurt And Cream	Sour		Fresh	Cooking Or Baking Ingredient	Low Nutritional Value	Animal Produce	
Activia Strawberry Gut Health Yoghurt 4x115g	Yoghurt And Cream	Sweet		Fresh	Animal Produce	Health Food Or Drink		
Aldi British Double Cream 300Ml	Yoghurt And Cream	Neutral		Low Nutritional Value	Cooking Or Baking Ingredient	Fresh	Animal Produce	
Vegan Fresh Soured Cream 142Ml	Yoghurt And Cream	Sour		Fresh	Cooking Or Baking Ingredient	Low Nutritional Value	Vegan	
Lindahls Kvarg Raspberry 150G Pot	Yoghurt And Cream	Sweet		Low Calorie Option	On The Go Food Or Drink	Animal Produce	Fresh	
Tesco 0% Fat Greek Style Yoghurt 500G	Yoghurt And Cream	Neutral		Animal Produce	Health Food Or Drink	Low Calorie Option	Fresh	
The Collective Plant-Based Yoghurt Alternative Raspberry 135G	Yoghurt And Cream	Sweet		Fresh	Health Food Or Drink	Vegan	On The Go Food Or Drink	
Muller Light Toffee Yoghurt 160G	Yoghurt And Cream	Sweet		Fresh	Health Food Or Drink	Low Calorie Option	Animal Produce	
Yeo Valley Organic Honey Greek Style Yoghurt 4 X100g	Yoghurt And Cream	Sweet		Health Food Or Drink	Organic Or Additive Free	Animal Produce	Fresh	
Graham's Protein 25G Passion Fruit & Mango Yoghurt 4x200G	Yoghurt And Cream	Sweet		Animal Produce	Fresh	Health Food Or Drink		
Muller Light Toffee Yoghurt 160G	Yoghurt And Cream	Sweet		Fresh	Health Food Or Drink	Low Calorie Option	Animal Produce	"""

In [4]:
lines = data_str.split('\n')

expanded_dataset = []
for line in lines:
    parts = line.split('\t')
    if len(parts) < 6:  # Ensure there's enough parts to include Level 3 and at least one tag
        continue

    line_item = parts[0]
    category = parts[1]  # Corrected to take the third column as "Category"
    flavours = parts[2:3]
    tags = parts[4:]  # Tags start from the fourth column onwards

    # Construct the dictionary for the current line
    line_dict = {
        "Line Item": line_item,
        "Category": category,
        "Flavours": [flavour for flavour in flavours if flavour.strip()],
        "Tags": [tag for tag in tags if tag.strip()]  # Only add non-empty tags, stripping any whitespace
    }
    
    # Add the constructed dictionary to the overall dataset
    expanded_dataset.append(line_dict)

# Assuming the first line is headers and should be ignored
expanded_dataset = expanded_dataset[1:]

[{'Line Item': 'Heinz By Nature Chicken Dinner 6 Month Plus 120G',
  'Category': 'Baby Food',
  'Flavours': ['Savoury'],
  'Tags': ['Health Food Or Drink',
   'Organic Or Additive Free',
   'Microwave Or Oven Ready']},
 {'Line Item': 'HiPP Organic Vegetables with Rice And Chicken Baby Food Jar 6+ Months 125g',
  'Category': 'Baby Food',
  'Flavours': ['Savoury'],
  'Tags': ['Health Food Or Drink', 'Microwave Or Oven Ready']},
 {'Line Item': 'SMA Pro 2 Follow-on Milk Powder, 6 mths+ 800g',
  'Category': 'Baby Food',
  'Flavours': ['Neutral'],
  'Tags': ['Health Food Or Drink', 'Microwave Or Oven Ready']},
 {'Line Item': 'Heinz Sweet Potato & Chicken Jar 120G',
  'Category': 'Baby Food',
  'Flavours': ['Sweet'],
  'Tags': ['Health Food Or Drink', 'Microwave Or Oven Ready']},
 {'Line Item': 'BabyBio Petit Pot Bonne Nuit Carottes Potimarron Riz (Dès 6 Mois) 2x200g',
  'Category': 'Baby Food',
  'Flavours': ['Savoury'],
  'Tags': ['Health Food Or Drink', 'Microwave Or Oven Ready']}]

In [ ]:
######################################
"""First try the fine tune with GPT3.5"""
######################################

In [5]:
#Create Jsonl structure for gpt3.5

# Function to convert the line items into the structured format
def gpt3_convert_items_to_chat_format(items):
    chat_sessions = []
    for item in items:
        session = {
            "messages": [
                {"role": "system", "content": "Using only the categories included in your fine-tuning training dataset, categorise and tag the item based on the product's nature. Max 6 Tags."},
                {"role": "user", "content": f"{item['Line Item']}"},
                {"role": "assistant", "content": f"Category: {item['Category']}; Flavours: {', '.join(item['Flavours'])}; Tags: {', '.join(item['Tags'])}"}
            ]
        }
        chat_sessions.append(session)
    return chat_sessions

# Convert the line items
gpt3_formatted_fine_tuning_data = gpt3_convert_items_to_chat_format(expanded_dataset)

[{'messages': [{'role': 'system',
    'content': "Using only the categories included in your fine-tuning training dataset, categorise and tag the item based on the product's nature. Max 6 Tags."},
   {'role': 'user',
    'content': 'Heinz By Nature Chicken Dinner 6 Month Plus 120G'},
   {'role': 'assistant',
    'content': 'Category: Baby Food; Flavours: Savoury; Tags: Health Food Or Drink, Organic Or Additive Free, Microwave Or Oven Ready'}]},
 {'messages': [{'role': 'system',
    'content': "Using only the categories included in your fine-tuning training dataset, categorise and tag the item based on the product's nature. Max 6 Tags."},
   {'role': 'user',
    'content': 'HiPP Organic Vegetables with Rice And Chicken Baby Food Jar 6+ Months 125g'},
   {'role': 'assistant',
    'content': 'Category: Baby Food; Flavours: Savoury; Tags: Health Food Or Drink, Microwave Or Oven Ready'}]},
 {'messages': [{'role': 'system',
    'content': "Using only the categories included in your fine-tuni

In [7]:
#Create datetime string to save data with
datetimenow = datetime.now().strftime('%d%m%y_%H%M')

In [8]:
# Save to JSONL file for gpt3
gpt3_train_file_path = rf"C:\Users\bened\OneDrive\Documents\Businesses\Relationship Predicting\Tesco Clubcards\3 - Raw Data Files\gpt3.5_train_{datetimenow}.jsonl"
with open(gpt3_train_file_path, 'w', encoding='utf-8') as file:
    for session in gpt3_formatted_fine_tuning_data:
        # Write each session as a JSON line
        file.write(json.dumps(session, ensure_ascii=False) + '\n')

In [10]:
# Initialize the OpenAI client with the API key
client = openai.Client(api_key=api_key)

# Upload the file for fine-tuning
gpt3_uploaded_file = client.files.create(
  file=open(gpt3_train_file_path, "rb"),
  purpose="fine-tune"
)
print(f"File uploaded successfully: {gpt3_uploaded_file.id}")

# Create a fine-tuning job using the uploaded file
gpt3_fine_tuning_job = client.fine_tuning.jobs.create(
  training_file=gpt3_uploaded_file.id,  # Use the file ID from the upload response
  model="gpt-3.5-turbo",  # Specify the GPT-3.5 model
  hyperparameters={
    "n_epochs": 6,
    "batch_size": 1
    # "learning_rate_multiplier": Optionally set if needed
  }
)
print(f"Fine-tuning job created: {gpt3_fine_tuning_job.id}")


File uploaded successfully: file-hauHoHFEtbmoXot0diDBWCRf
Fine-tuning job created: ftjob-NMGUoI8theqkfg7Jxoa2o89x


In [11]:
#cehck status of fine tune
gpt3_status = client.fine_tuning.jobs.retrieve(gpt3_fine_tuning_job.id)

# Formatting the output
gpt3_details = (
    f"Fine Tuning Job Details:\n"
    f"Job ID: {gpt3_status.id}\n"
    f"Status: {gpt3_status.status}\n"
    f"Created At: {gpt3_status.created_at}\n"
    f"Base Model: {gpt3_status.model}\n"
    f"Training File ID: {gpt3_status.training_file}\n"
    f"Validation File ID: {gpt3_status.validation_file or 'None'}\n"
    f"Error: {gpt3_status.error or 'No errors reported'}\n"
    f"Trained Tokens: {gpt3_status.trained_tokens or 'Not available'}\n"
    f"Result Files: {', '.join(gpt3_status.result_files) if gpt3_status.result_files else 'No result files'}\n"
    f"Hyperparameters: Epochs={gpt3_status.hyperparameters.n_epochs}, Batch Size={gpt3_status.hyperparameters.batch_size}, Learning Rate Multiplier={gpt3_status.hyperparameters.learning_rate_multiplier}\n"
)

print(gpt3_details)

Fine Tuning Job Details:
Job ID: ftjob-NMGUoI8theqkfg7Jxoa2o89x
Status: running
Created At: 1716900851
Base Model: gpt-3.5-turbo-0125
Training File ID: file-hauHoHFEtbmoXot0diDBWCRf
Validation File ID: None
Error: Error(code=None, message=None, param=None)
Trained Tokens: Not available
Result Files: No result files
Hyperparameters: Epochs=6, Batch Size=1, Learning Rate Multiplier=2



In [ ]:
############################
"""Now to fine tune Davinci"""
############################

In [ ]:
#Create Jsonl structure for Davinci

def davinci_convert_items_to_finetuning_format(items):
    fine_tuning_data = []
    for item in items:
        # Construct the prompt as it would appear in a chat
        prompt = f"Categorize this item based on the product's nature: {item['Line Item']}"

        # Construct the expected completion, which should follow directly after the prompt
        completion = f"Category: {item['Category']}; Flavour: {item["Flavour"]}; Tags: {', '.join(item['Tags'])}"

        # Append the structured data to the fine-tuning data list
        fine_tuning_data.append({"prompt": prompt, "completion": completion})

    return fine_tuning_data

# Convert the line items to fine-tuning format
davinci_formatted_fine_tuning_data = davinci_convert_items_to_finetuning_format(expanded_dataset)

# Printing the structured fine-tuning data to verify the output
davinci_formatted_fine_tuning_data[:4]


# Save to JSONL file for Davinci
davinci_train_file_path = rf"C:\Users\bened\OneDrive\Documents\Businesses\Relationship Predicting\Tesco Clubcards\3 - Raw Data Files\davinci_train_{datetimenow}.jsonl"
with open(davinci_train_file_path, 'w', encoding='utf-8') as file:
    for session in davinci_formatted_fine_tuning_data:
        # Write each session as a JSON line
        file.write(json.dumps(session, ensure_ascii=False) + '\n')

In [1]:
try:
    # Upload file and create fine-tuning job
    davinci_uploaded_file = client.files.create(
        file=open(davinci_train_file_path, "rb"),
        purpose="fine-tune"
    )
    print(f"File uploaded successfully: {davinci_uploaded_file.id}")

    # Create fine-tuning job
    davinci_fine_tuning_job = client.fine_tuning.jobs.create(
        training_file=davinci_uploaded_file.id,
        model="davinci-002",
        hyperparameters={
            "n_epochs": 5,
            "batch_size": 1 
            # "learning_rate_multiplier":
        }
    )
    print(f"Fine-tuning job created: {davinci_fine_tuning_job.id}")

except openai.Error as e:
    print(f"An error occurred: {e}")


NameError: name 'openai' is not defined

In [ ]:
#check status of fine tune
davinci_status = client.fine_tuning.jobs.retrieve(davinci_fine_tuning_job.id)

# Formatting the output
davinci_details = (
    f"Fine Tuning Job Details:\n"
    f"Job ID: {davinci_status.id}\n"
    f"Status: {davinci_status.status}\n"
    f"Created At: {davinci_status.created_at}\n"
    f"Base Model: {davinci_status.model}\n"
    f"Training File ID: {davinci_status.training_file}\n"
    f"Validation File ID: {davinci_status.validation_file or 'None'}\n"
    f"Error: {davinci_status.error or 'No errors reported'}\n"
    f"Trained Tokens: {davinci_status.trained_tokens or 'Not available'}\n"
    f"Result Files: {', '.join(davinci_status.result_files) if davinci_status.result_files else 'No result files'}\n"
    f"Hyperparameters: Epochs={davinci_status.hyperparameters.n_epochs}, Batch Size={davinci_status.hyperparameters.batch_size}, Learning Rate Multiplier={davinci_status.hyperparameters.learning_rate_multiplier}\n"
)

print(davinci_details)


In [ ]:
##########################
"""Now they're running, lets see how old jobs are doing"""
##########################

In [ ]:
#Section to get info about current and past fine tuning jobs
import datetime

# Retrieve a list of fine-tuning jobs
jobs_page = client.fine_tuning.jobs.list()

def format_timestamp(ts):
    """Convert Unix timestamp to a human-readable date and time string."""
    return datetime.datetime.utcfromtimestamp(ts).strftime('%d/%m/%Y %H:%M')

# Formatting and printing each job in the page
for job in jobs_page.data:
    created_at_formatted = format_timestamp(job.created_at)
    details = (
        f"Fine Tuning Job Details:\n"
        f"Job ID: {job.id}\n"
        f"Status: {job.status}\n"
        f"Created At: {created_at_formatted}\n"
        f"Base Model: {job.model}\n"
        f"Training File ID: {job.training_file}\n"
        f"Validation File ID: {job.validation_file or 'None'}\n"
        f"Error: {job.error or 'No errors reported'}\n"
        f"Trained Tokens: {job.trained_tokens or 'Not available'}\n"
        f"Result Files: {', '.join(job.result_files) if job.result_files else 'No result files'}\n"
        f"Hyperparameters: Epochs={job.hyperparameters.n_epochs if job.hyperparameters else 'N/A'}, "
        f"Batch Size={job.hyperparameters.batch_size if job.hyperparameters else 'N/A'}, "
        f"Learning Rate Multiplier={job.hyperparameters.learning_rate_multiplier if job.hyperparameters else 'N/A'}\n"
    )
    print(details)

# Checking if there's another page
if jobs_page.has_next_page():
    print("There are more fine-tuning jobs to fetch.")
else:
    print("No more fine-tuning jobs to fetch.")


In [52]:
###########################
"""Now to use our trained models"""
###########################

'Now to use our trained models'

In [74]:
#Section to test categorising a single input

gpt3_model = "ft:gpt-3.5-turbo-0125:personal::9TsbbInd"

item_name = "halloumi"

#Prompting fine tuned 3.5 model
completion = client.chat.completions.create(
    model=gpt3_model,
    messages=[
        {"role": "system", "content": "Categorise and tag the item based on the product's nature, using only predefined options."},
        {"role": "user", "content": item_name}
    ],
    #max_tokens=150,
    temperature=0,  # Lower temperature for more deterministic responses
    top_p=0,#Probability selection of next token (0 for less diverse, 1 for max diversity)
    frequency_penalty=0,  # To reduce repetition (-2 no penalty, 2 avoids repetition)
    presence_penalty=-0.5,  # To encourage variety in mentioning new topics (-2 sticks to old topics, 2 gets new ones)
    # stop=["\n", "<|endoftext|>"]  # Stop at end of message or when the model thinks it's the end of the text
)
print(completion.choices[0].message.content)

Category: Cheese And Alternatives; Flavours: Savoury; Tags: Mediterranean Food, Low Nutritional Value, Fresh, Cooking Or Baking Ingredient


In [ ]:
davinci_model = "ft:davinci-002:personal::9MijbrRq"

# #Prompt fine tuned davinci model
completion = client.completions.create(
    model=davinci_model,
    prompt="Categorize this item based on the product's nature: Tesco Loose Red Peppers(C)",
    # max_tokens=30,
    temperature=0.1,  # Lower temperature for more deterministic responses
    top_p=0.2,#Probability selection of next token (0 for less diverse, 1 for max diversity)
    frequency_penalty=-1,  # To reduce repetition (-2 no penalty, 2 avoids repetition)
    presence_penalty=0,  # To encourage variety in mentioning new topics (-2 sticks to old topics, 2 gets new ones)
    stop=["\n", "<|endoftext|>"]  # Stop at end of message or when the model thinks it's the end of the text
)
print(completion.choices[0].text.strip())


In [ ]:
########################
#Info on past models
"""

ft:gpt-3.5-turbo-0125:personal::9TYWfkwo 6 epoch, 1 batch size, 2 LRM. Improved categories, mis-identification of similar ones (eclairs into confectionary) so more data needed. Also imporve specific cats like baby food Best yet.
Small issues in data, like had a "Fermented Food" tag & spelling mistakes. CHECK DATA INTEGIRTY AND DO IT AGAIN 27/05/24

ft:gpt-3.5-turbo-0125:personal::9R4ha0SG  5 epoch, 1 batch size, 2 LRM. Similar to last one (maybe not as good) but new categories added - 20/05/24

"ft:gpt-3.5-turbo-0125:personal::9OBc7Pa4", 5 epoch, 1 batch size, 2 LRM. New winner, better data, refined categories

ft:gpt-3.5-turbo-0125:personal::9O4yIFFA,  5 epoch, 1 batch size, 2 LRM. - Trained with 1, A, ! - coding system. Coding system worked, but categoriser is junk.

ft:davinci-002:personal::9MijbrRq, created 08/05/24, 5 epoch, 1 batch size, 16 LRM. - OK

ft:gpt-3.5-turbo-0125:personal::9MipZzPn, created 08/05/24, 5 epoch, 1 batch size, 2 LRM. - BEST SO FAR


"""

In [ ]:
"""
METHOD TO IMPROVE

Get a huge database of all items (banana, soup, salmon, sticky toffe - super granular)
Categorise these items in a big mapping file (maybe exists on open food facts?)
GPT to match the item to the granular item, which is a closer match than the category
###CHANGE PROMPT TO THIS         {"role": "system", "content": "Using only the categories, tags, and flavours from the training dataset, categorize and tag the item based on its nature. Do not create or infer new categories or tags."},


"""